In [40]:
import os
import glob
import pandas as pd
import numpy as np
import math as m
from collections import defaultdict

In [2]:
#Used for easier inspection of the columns and data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 300)

In [3]:
#File path for input raw data
data_path = os.path.join(os.path.dirname(os.getcwd()), 'Data2')

#File for reference file
ref = pd.read_csv(os.path.join(os.path.dirname(os.getcwd()), 'Reference Files', 'Battlerite_Crosswalk.csv'))
ref = ref[['Battlerite Type', 'Value']]

### Data gathering for data inside "Data2" done at https://github.com/Skywind555/Battlerite/blob/master/Get_Data_V2.py

### Data gathering for data inside "Reference Files" done at https://github.com/Skywind555/Battlerite/blob/master/Get_Reference_Files_V2.py

### Keep in mind that the API has been dead since around mid-July 2019 and the two scripts above are no longer functional.

In [4]:
#Read in all data in data2 folder
all_files = glob.glob(os.path.join(data_path, '**/**/**/Jul*.csv'))
list_dfs = [pd.read_csv(f) for f in all_files]
df = pd.concat(list_dfs, ignore_index = True)

### Each row in the data represents one round for one player in one game.

In [5]:
df.head(4)

,Game_ID,Patch_Version,Date,Map,Match_Type,Ranking_Type,Server_Type,Solo_Queue,Season_ID,Region,City,Latitude,Longitude,Limited_MM,Player_Type,Name,User_ID,Champion,Role,Forfeited,Group,User_Leave_Type,Left_During_Round,AFK_Detection,League,Division,Division_Rating,Total_Time_Played,Account_Level,Champion_Level,Champion_Time_Played,Round,Kills,Deaths,Assists,Damage,Damage_Received,Protection,Protection_Received,Control,Control_Received,Energy_Used,Energy_Gained,Abilities_Used,Ultimates,Num_R,Num_EX1,Num_EX2,Num_Energy_Used,Orb_Kills,First_Orb,MVP,Round_Length,Round_Won,Total_Score,Highest_Score,Health_Shards,Energy_Shards,DeathOrbs,Team_Orbs,Enemy_Orbs,Time_Alive,Premade,Queue_Time,Ping,Avatar,Title,Outfit Rarity,Outfit,Attachment Rarity,Attachment,Pose Rarity,Pose,Mount Rarity,Mount,Battlerite 1,Battlerite 2,Battlerite 3,Battlerite 4,Battlerite 5,Battlerites,Team_Comp,Team_Roles,Ally1Role,Ally1Champion,Ally1ChampionPlayed,Ally1TotalPlayed,Ally1TotalScore,Ally1DamageReceived,Ally1TimeAlive,Ally1League,Ally1Ping,Ally1HealthShards,Ally1EnergyShards,Ally1NumEnergyUsed,Ally1HighestScore,Ally1Battlerite1,Ally1Battlerite2,Ally1Battlerite3,Ally1Battlerite4,Ally1Battlerite5,Ally1Battlerites,Ally2Role,Ally2Champion,Ally2ChampionPlayed,Ally2TotalPlayed,Ally2TotalScore,Ally2DamageReceived,Ally2TimeAlive,Ally2League,Ally2Ping,Ally2HealthShards,Ally2EnergyShards,Ally2NumEnergyUsed,Ally2HighestScore,Ally2Battlerite1,Ally2Battlerite2,Ally2Battlerite3,Ally2Battlerite4,Ally2Battlerite5,Ally2Battlerites,Enemy_Comp,Enemy_Roles,Enemy_Total_HealthShards,Enemy_Total_EnergyShards,Enemy1Role,Enemy1Champion,Enemy1ChampionPlayed,Enemy1TotalPlayed,Enemy1TotalScore,Enemy1DamageReceived,Enemy1TimeAlive,Enemy1League,Enemy1Ping,Enemy1NumEnergyUsed,Enemy1HighestScore,Enemy1Battlerite1,Enemy1Battlerite2,Enemy1Battlerite3,Enemy1Battlerite4,Enemy1Battlerite5,Enemy1Battlerites,Enemy2Role,Enemy2Champion,Enemy2ChampionPlayed,Enemy2TotalPlayed,Enemy2TotalScore,Enemy2DamageReceived,Enemy2TimeAlive,Enemy2League,Enemy2Ping,Enemy2Ping.1,Enemy2NumEnergyUsed,Enemy2HighestScore,Enemy3Role,Enemy3Champion,Enemy3ChampionPlayed,Enemy3TotalPlayed,Enemy3TotalScore,Enemy3DamageReceived,Enemy3TimeAlive,Enemy3League,Enemy2Ping.2,Enemy2Ping.3,Enemy3NumEnergyUsed,Enemy3HighestScore,Enemy2Battlerite1,Enemy2Battlerite2,Enemy2Battlerite3,Enemy2Battlerite4,Enemy2Battlerite5,Enemy2Battlerites,Enemy3Battlerite1,Enemy3Battlerite2,Enemy3Battlerite3,Enemy3Battlerite4,Enemy3Battlerite5,Enemy3Battlerites
0,3C034044B8BE45E9BA4E3C02C7282FA2,67104,2019-07-10T09:17:16Z,MISTY WOODS - DAY,LEAGUE3V3,RANKED,QUICK3V3,1,14,spain,Madrid,40.4168,-3.7038,0.0,PLAYER,zoek11,1085583168129654784,BAKKO,MELEE,False,1,NORMAL,-1,False,Gold,3,90,484010.0,23,20,466541.0,1,0.0,0.0,0.0,853.0,473.19998,84.50000,278.30002,88.0,124.0,275.0,311.0,75.0,2.0,3.0,0.0,0.0,3.0,1.0,1.0,False,84.0,1,1025.50000,0,3.0,2.0,0.0,3.0,0.0,-93.0,0.0,207.929530,15.0,Lunar_20,Contender,Legendary,BLACK BANNER,Legendary,ARMORED BONE BREAKER,Common,DEFAULT POSE,Epic,FREE-RANGE EGG,WARLORD'S AXE,THE RED AXE,ADRENALINE RUSH,MOBILE DEFENCE,RAGING RAM,"WARLORD'S AXE, THE RED AXE, ADRENALINE RUSH, MOBILE DEFENCE, RAGING RAM","BAKKO, PEARL, CROAK","MELEE, SUPPORT, MELEE",SUPPORT,PEARL,502640.0,1066814.0,1273.16675,351.56665,-93.0,Platinum,69.0,0.0,2.0,6.0,0,SPLASH DAMAGE,FOUNTAIN,SPRING WATER,OCEAN TIDE,GUSH,"SPLASH DAMAGE, FOUNTAIN, SPRING WATER, OCEAN TIDE, GUSH",MELEE,CROAK,371202.0,2302387.0,1305.46664,518.64990,-93.0,Silver,51.0,0.0,2.0,6.0,1.0,MENDING TOXIN,TIME TO STRIKE,NOXIOUS REACTION,MOMENTUM,FROG FRENZY,"MENDING TOXIN, TIME TO STRIKE, NOXIOUS REACTION, MOMENTUM, FROG FRENZY","ULRIC, THORN, BLOSSOM","SUPPORT, MELEE, SUPPORT",3.0,0.0,SUPPORT,ULRIC,15847.0,465134.0,882.40598,626.0,66.0,Gold,65.0,3.0,0,DIVINE ABUNDANCE,FAITH,HALO,HOLY FIRE,DIVINE PROTECTION,"DIVINE ABUNDANCE, FAITH, HALO, HOLY FIRE, DIVINE PROTECTION",MELEE,THORN,85714.0,1895920.0,1253.04070,811.0,83.0,Platinum,27.0,78.0,2.0,0,SUPPORT,BLOSSOM,13620.0,217667.0,790.76,598.0,6

### In order to gather more insight from the data and have the ability to build a model, the data needs to be cleaned and transformed.

In [6]:
game_bots = []
game_leaver = []
role = []
ally1role = []
ally2role = []
enemy1role = []
enemy2role = []
enemy3role = []
for index, row in df.iterrows():
    
    #Determine if game ID has bots
    #Based on analysis / data cleaning done for the R Shiny dashboard, these values imply bot
    if pd.isnull(row['Outfit']) or pd.isnull(row['Mount']):
        if row['Game_ID'] not in game_bots:
            game_bots.append(row['Game_ID'])
            
    #Determine if there are afk/forfeit/leaver in game ID
    if row['Forfeited'] == True or row['AFK_Detection'] == True:
        if row['Game_ID'] not in game_leaver:
            game_leaver.append(row['Game_ID'])
            
    #Role detailed add Support (Melee) and Support (Range)
    if row['Champion'] in ['ULRIC', 'SIRIUS']:
        role.append('SUPPORT (MELEE)')
    elif row['Role'] == 'SUPPORT':
        role.append('SUPPORT (RANGED)')
    else:
        role.append(row['Role'])
        
    if row['Ally1Champion'] in ['ULRIC', 'SIRIUS']:
        ally1role.append('SUPPORT (MELEE)')
    elif row['Ally1Role'] == 'SUPPORT':
        ally1role.append('SUPPORT (RANGED)')
    else:
        ally1role.append(row['Ally1Role'])
        
    if row['Ally2Champion'] in ['ULRIC', 'SIRIUS']:
        ally2role.append('SUPPORT (MELEE)')
    elif row['Ally2Role'] == 'SUPPORT':
        ally2role.append('SUPPORT (RANGED)')
    else:
        ally2role.append(row['Ally2Role'])
        
    if row['Enemy1Champion'] in ['ULRIC', 'SIRIUS']:
        enemy1role.append('SUPPORT (MELEE)')
    elif row['Enemy1Role'] == 'SUPPORT':
        enemy1role.append('SUPPORT (RANGED)')
    else:
        enemy1role.append(row['Enemy1Role'])
        
    if row['Enemy2Champion'] in ['ULRIC', 'SIRIUS']:
        enemy2role.append('SUPPORT (MELEE)')
    elif row['Enemy2Role'] == 'SUPPORT':
        enemy2role.append('SUPPORT (RANGED)')
    else:
        enemy2role.append(row['Enemy2Role'])
        
    if row['Enemy3Champion'] in ['ULRIC', 'SIRIUS']:
        enemy3role.append('SUPPORT (MELEE)')
    elif row['Enemy3Role'] == 'SUPPORT':
        enemy3role.append('SUPPORT (RANGED)')
    else:
        enemy3role.append(row['Enemy3Role'])
        
#Add detailed support roles
df['Role_Detail'] = role
df['Ally1RoleDetail'] = ally1role
df['Ally2RoleDetail'] = ally2role
df['Enemy1RoleDetail'] = enemy1role
df['Enemy2RoleDetail'] = enemy2role
df['Enemy3RoleDetail'] = enemy3role

def add_ally_role_detail(x):
    '''
    Depending on if 3v3 or 2v2, concats all detailed ally roles to one column.
    '''
    if x['Server_Type'] == 'QUICK3V3':
        return (x['Role_Detail'] + ', ' + x['Ally1RoleDetail'] + ', ' + x['Ally2RoleDetail'])
    else:
        return (x['Role_Detail'] + ', ' + x['Ally1RoleDetail'])
    
def add_enemy_role_detail(x):
    '''
    Depending on if 3v3 or 2v2, concats all detailed enemy roles to one column
    '''
    if x['Server_Type'] == 'QUICK3V3':
        return (x['Enemy1RoleDetail'] + ', ' + x['Enemy2RoleDetail'] + ', ' + x['Enemy3RoleDetail'])
    else:
        return (x['Enemy1RoleDetail'] + ', ' + x['Enemy2RoleDetail'])

df['TeamRolesDetail'] = df.apply(add_ally_role_detail, axis = 1)
df['EnemyRolesDetail'] = df.apply(add_enemy_role_detail, axis = 1)

In [7]:
#Filter out bot games
df = df[~df['Game_ID'].isin(game_bots)].reset_index(drop = True)

#Filter out afk/forfeit/leaver games
df = df[~df['Game_ID'].isin(game_leaver)].reset_index(drop = True)

In [8]:
def standardize_values_unique(col):
    '''
    Helper function to clean values in text columns so that each combination of values appear in the same order, avoiding
    redundancy. i.e. if 'A, B, C' and 'C, A, B' appear in the list the function aims to convert all values to the format
    'A, B, C' for all values containing A, B, and C as a group.
    '''
    group_values = []
    standardized_values = []
    for row in df[col]:
        list_values = row.split(', ')
        group_value = set(list_values)
        if not group_value in group_values:
            group_values.append(group_value)        
        standardized_values.append(', '.join(list(group_value)))
        
    df[col] = standardized_values

In [9]:
def standardize_values_dupe(data, col):
    '''
    Helper function to clean values in text columns similar to standardize_values_unique except this version
    will work on duplicate values. i.e. if 'A, A, B' and 'B, A, A' appear in the list the function aims to convert
    all values to a single format 'A, A, B' and 'A, A, B' in this case.
    '''
    group_values = []
    standardized_values = []
    for row in data[col]:
        list_values = row.split(', ')
        new_dict = defaultdict(int)
        for v in list_values:
            new_dict[v] += 1
        match_exist = False
        if len(group_values) > 0:
            for group in group_values:
                group_list = group.split(', ')
                group_dict = defaultdict(int)
                for v in group_list:
                    group_dict[v] += 1
                    
                if group_dict == new_dict:
                    standardized_values.append(group)
                    match_exist = True
                    break
            
            if not match_exist:
                group_values.append(row)
                standardized_values.append(row)
            
        else:
            group_values.append(row)
            standardized_values.append(row)
        
    data[col] = standardized_values

In [10]:
standardize_values_dupe(df, 'Team_Roles')
standardize_values_dupe(df, 'Enemy_Roles')
standardize_values_dupe(df, 'TeamRolesDetail')
standardize_values_dupe(df, 'EnemyRolesDetail')

standardize_values_unique('Battlerites')
standardize_values_unique('Team_Comp')
standardize_values_unique('Ally1Battlerites')
standardize_values_unique('Ally2Battlerites')
standardize_values_unique('Enemy_Comp')
standardize_values_unique('Enemy1Battlerites')
standardize_values_unique('Enemy2Battlerites')
standardize_values_unique('Enemy3Battlerites')

In [11]:
def add_battlerite_type(col, new_column_name):
    '''
    Helper function to add battlerite type to the columns that include a set of 5 battlerites.
    '''
   
    br_types_all = []
    for row in df[col]:
        if row != 'None':
            list_values = row.split(', ')
            br_types = []


            for val in list_values:
                match = ref[ref['Value'] == val]
                br_types.append(match['Battlerite Type'].values[0])
            br_types_all.append(', '.join(br_types))
        else:
            br_types_all.append('None')
        
    df[new_column_name] = br_types_all

In [12]:
add_battlerite_type('Battlerites', 'BattleritesType')
add_battlerite_type('Ally1Battlerites', 'Ally1BattleritesType')
add_battlerite_type('Ally2Battlerites', 'Ally2BattleritesType')
add_battlerite_type('Enemy1Battlerites', 'Enemy1BattleritesType')
add_battlerite_type('Enemy2Battlerites', 'Enemy2BattleritesType')
add_battlerite_type('Enemy3Battlerites', 'Enemy3BattleritesType')

#Standardize values for order differences for same set
standardize_values_dupe(df, 'BattleritesType')
standardize_values_dupe(df, 'Ally1BattleritesType')
standardize_values_dupe(df, 'Ally2BattleritesType')
standardize_values_dupe(df, 'Enemy1BattleritesType')
standardize_values_dupe(df, 'Enemy2BattleritesType')
standardize_values_dupe(df, 'Enemy3BattleritesType')

In [13]:
#Remove useless columns and columns created in error
df = df.drop(['Patch_Version', 'Latitude', 'Longitude', 'City', 'Left_During_Round', 'User_Leave_Type',
              'Forfeited', 'AFK_Detection','Enemy2Ping.1', 'Enemy2Ping.2', 'Date', 'Season_ID',
              'Player_Type', 'Division_Rating', ] , axis = 1)

df = df.rename(columns = {'Enemy2Ping.3':'Enemy3Ping'})

In [14]:
#Create total score / damage received ratio for allies and enemies
for selection in ['', 'Ally1', 'Ally2', 'Enemy1', 'Enemy2', 'Enemy3']:
    if selection == '':
        df[selection + 'ScoreDamageRatio'] = df['Total_Score'] / df['Damage_Received']
    else:
        df[selection + 'ScoreDamageRatio'] = df[selection + 'TotalScore'] / df[selection + 'DamageReceived']

#Inf values appear when division by 0
df_inf = df[(df['ScoreDamageRatio'] == np.inf) | (df['Ally1ScoreDamageRatio'] == np.inf) |
    (df['Ally2ScoreDamageRatio'] == np.inf) | (df['Enemy1ScoreDamageRatio'] == np.inf) |
    (df['Enemy2ScoreDamageRatio'] == np.inf) | (df['Enemy3ScoreDamageRatio'] == np.inf)]
 
#For these values assume that they took 1 damage and set equal to score
for index, row in df_inf.iterrows():
    for selection in ['', 'Ally1', 'Ally2', 'Enemy1', 'Enemy2', 'Enemy3']:
        if selection == '':
            df.loc[index, 'ScoreDamageRatio'] = row['Total_Score']
        else:
            df.loc[index, selection+'ScoreDamageRatio'] = row[selection+'TotalScore']

In [15]:
def convert_negatives(x):
    '''
    Convert negative values into positive values. Negative values exist because if someone never dies
    in a round, the time_alive variable shows up as negative.
    '''
    if x < 0:
        return (x*-1)
    else:
        return x

In [16]:
#Convert negatives to positives
df['Time_Alive'] = df['Time_Alive'].apply(convert_negatives)

#Calculate Score per second based on Total score and time alive
df['ScorePerSecond'] = df['Total_Score']/df['Time_Alive']
    
#Calculate Score per second based on Total score and time alive for allies and enemies
for selection in ['Ally1', 'Ally2', 'Enemy1', 'Enemy2', 'Enemy3']:
    #Convert negatives to positives
    df[selection + 'TimeAlive'] = df[selection + 'TimeAlive'].apply(convert_negatives)
    #Calculate Score per second based on Total score and time alive for allies and enemies
    df[selection + 'ScorePerSecond'] = df[selection + 'TotalScore'] / df[selection + 'TimeAlive']
    
#Add Team total health / energy orbs
df['TeamTotalHealthShards'] = df['Health_Shards'] + df['Ally1HealthShards'] + df['Ally2HealthShards'].fillna(0)
df['TeamTotalEnergyShards'] = df['Energy_Shards'] + df['Ally1EnergyShards'] + df['Ally2EnergyShards'].fillna(0)

In [17]:
#Keep current copy of df for reference
baseline_df = df.copy()

In [18]:
'''
The following adds additional column information about allies/enemies in the same game 
present in other rows of same Game ID

'''
    
FinalAlly1NumR = []
FinalAlly1NumEX1 = []
FinalAlly1NumEX2 = []
FinalAlly1NumUltimate = []
FinalAlly1HealthShards = []
FinalAlly1EnergyShards = []
FinalAlly1DeathOrbs = []
FinalAlly1Division = []
FinalAlly1Damage = []
FinalAlly1Protection = []
FinalAlly1ProtectionReceived = []
FinalAlly1Control = []
FinalAlly1ControlReceived = []
FinalAlly1EnergyUsed = []
FinalAlly1EnergyGained = []
FinalAlly1Deaths = []
FinalAlly2NumR = []
FinalAlly2NumEX1 = []
FinalAlly2NumEX2 = []
FinalAlly2NumUltimate = []
FinalAlly2DeathOrbs = []
FinalAlly2HealthShards = []
FinalAlly2EnergyShards = []
FinalAlly2Division = []
FinalAlly2Damage = []
FinalAlly2Protection = []
FinalAlly2ProtectionReceived = []
FinalAlly2Control = []
FinalAlly2ControlReceived = []
FinalAlly2EnergyUsed = []
FinalAlly2EnergyGained = []
FinalAlly2Deaths = []
FinalEnemy1NumR = []
FinalEnemy1NumEX1 = []
FinalEnemy1NumEX2 = []
FinalEnemy1NumUltimate = []
FinalEnemy1DeathOrbs = []
FinalEnemy1HealthShards = []
FinalEnemy1EnergyShards = []
FinalEnemy1Division = []
FinalEnemy1Damage = []
FinalEnemy1Protection = []
FinalEnemy1ProtectionReceived = []
FinalEnemy1Control = []
FinalEnemy1ControlReceived = []
FinalEnemy1EnergyUsed = []
FinalEnemy1EnergyGained = []
FinalEnemy1Deaths = []
FinalEnemy2NumR = []
FinalEnemy2NumEX1 = []
FinalEnemy2NumEX2 = []
FinalEnemy2NumUltimate = []
FinalEnemy2DeathOrbs = []
FinalEnemy2HealthShards = []
FinalEnemy2EnergyShards = []
FinalEnemy2Division = []
FinalEnemy2Damage = []
FinalEnemy2Protection = []
FinalEnemy2ProtectionReceived = []
FinalEnemy2Control = []
FinalEnemy2ControlReceived = []
FinalEnemy2EnergyUsed = []
FinalEnemy2EnergyGained = []
FinalEnemy2Deaths = []
FinalEnemy3NumR = []
FinalEnemy3NumEX1 = []
FinalEnemy3NumEX2 = []
FinalEnemy3NumUltimate = []
FinalEnemy3DeathOrbs = []
FinalEnemy3HealthShards = []
FinalEnemy3EnergyShards = []
FinalEnemy3Division = []
FinalEnemy3Damage = []
FinalEnemy3Protection = []
FinalEnemy3ProtectionReceived = []
FinalEnemy3Control = []
FinalEnemy3ControlReceived = []
FinalEnemy3EnergyUsed = []
FinalEnemy3EnergyGained = []
FinalEnemy3Deaths = []
bad_game_ids = []

for i in range(len(df['Game_ID'].unique())):
    
    df = baseline_df.copy()
    subset_game_id = df['Game_ID'].unique()[i]
    df = df[df['Game_ID'] == subset_game_id].reset_index(drop = True)

    Ally1NumR = []
    Ally1NumEX1 = []
    Ally1NumEX2 = []
    Ally1NumUltimate = []
    Ally1DeathOrbs = []
    Ally1HealthShards = []
    Ally1EnergyShards = []
    Ally1Division = []
    Ally1Damage = []
    Ally1Protection = []
    Ally1ProtectionReceived = []
    Ally1Control = []
    Ally1ControlReceived = []
    Ally1EnergyUsed = []
    Ally1EnergyGained = []
    Ally1Deaths = []
    Ally2NumR = []
    Ally2NumEX1 = []
    Ally2NumEX2 = []
    Ally2NumUltimate = []
    Ally2DeathOrbs = []
    Ally2HealthShards = []
    Ally2EnergyShards = []
    Ally2Division = []
    Ally2Damage = []
    Ally2Protection = []
    Ally2ProtectionReceived = []
    Ally2Control = []
    Ally2ControlReceived = []
    Ally2EnergyUsed = []
    Ally2EnergyGained = []
    Ally2Deaths = []
    Enemy1NumR = []
    Enemy1NumEX1 = []
    Enemy1NumEX2 = []
    Enemy1NumUltimate = []
    Enemy1DeathOrbs = []
    Enemy1HealthShards = []
    Enemy1EnergyShards = []
    Enemy1Division = []
    Enemy1Damage = []
    Enemy1Protection = []
    Enemy1ProtectionReceived = []
    Enemy1Control = []
    Enemy1ControlReceived = []
    Enemy1EnergyUsed = []
    Enemy1EnergyGained = []
    Enemy1Deaths = []
    Enemy2NumR = []
    Enemy2NumEX1 = []
    Enemy2NumEX2 = []
    Enemy2NumUltimate = []
    Enemy2HealthShards = []
    Enemy2EnergyShards = []
    Enemy2DeathOrbs = []
    Enemy2Division = []
    Enemy2Damage = []
    Enemy2Protection = []
    Enemy2ProtectionReceived = []
    Enemy2Control = []
    Enemy2ControlReceived = []
    Enemy2EnergyUsed = []
    Enemy2EnergyGained = []
    Enemy2Deaths = []
    Enemy3NumR = []
    Enemy3NumEX1 = []
    Enemy3NumEX2 = []
    Enemy3NumUltimate = []
    Enemy3DeathOrbs = []
    Enemy3HealthShards = []
    Enemy3EnergyShards = []
    Enemy3Division = []
    Enemy3Damage = []
    Enemy3Protection = []
    Enemy3ProtectionReceived = []
    Enemy3Control = []
    Enemy3ControlReceived = []
    Enemy3EnergyUsed = []
    Enemy3EnergyGained = []
    Enemy3Deaths = []

    for index, row in df.iterrows():
        round_df = df[df['Round'] == row['Round']]
        Enemy3 = False
        for index2, row2 in round_df.iterrows():
            #3v3
            if len(round_df) == 6:
                #Draw rounds
                if sum(round_df['Round_Won']) == 0:

                    if row2['Name'] != row['Name']:
                        #Ensure same team member 1 and not enemy team same champion
                        if row2['Champion'] == row['Ally1Champion'] and \
                        row2['Total_Time_Played'] == row['Ally1TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Ally1ChampionPlayed']:
                            Ally1NumR.append(row2['Num_R'])
                            Ally1NumEX1.append(row2['Num_EX1'])
                            Ally1NumEX2.append(row2['Num_EX2'])
                            Ally1NumUltimate.append(row2['Ultimates'])
                            Ally1DeathOrbs.append(row2['DeathOrbs'])
                            Ally1HealthShards.append(row2['Health_Shards'])
                            Ally1EnergyShards.append(row2['Energy_Shards'])
                            Ally1Division.append(row2['Division'])
                            Ally1Damage.append(row2['Damage'])
                            Ally1Protection.append(row2['Protection'])
                            Ally1ProtectionReceived.append(row2['Protection_Received'])
                            Ally1Control.append(row2['Control'])
                            Ally1ControlReceived.append(row2['Control_Received'])
                            Ally1EnergyUsed.append(row2['Energy_Used'])
                            Ally1EnergyGained.append(row2['Energy_Gained'])
                            Ally1Deaths.append(row2['Deaths'])

                        #Ensure same team member 2 and not enemy team same champion
                        elif row2['Champion'] == row['Ally2Champion'] and \
                        row2['Total_Time_Played'] == row['Ally2TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Ally2ChampionPlayed']:

                            Ally2NumR.append(row2['Num_R'])
                            Ally2NumEX1.append(row2['Num_EX1'])
                            Ally2NumEX2.append(row2['Num_EX2'])
                            Ally2NumUltimate.append(row2['Ultimates'])
                            Ally2DeathOrbs.append(row2['DeathOrbs'])
                            Ally2HealthShards.append(row2['Health_Shards'])
                            Ally2EnergyShards.append(row2['Energy_Shards'])
                            Ally2Division.append(row2['Division'])
                            Ally2Damage.append(row2['Damage'])
                            Ally2Protection.append(row2['Protection'])
                            Ally2ProtectionReceived.append(row2['Protection_Received'])
                            Ally2Control.append(row2['Control'])
                            Ally2ControlReceived.append(row2['Control_Received'])
                            Ally2EnergyUsed.append(row2['Energy_Used'])
                            Ally2EnergyGained.append(row2['Energy_Gained'])
                            Ally2Deaths.append(row2['Deaths'])

                        #Ensure same enemy 1 match
                        elif row2['Champion'] == row['Enemy1Champion'] and \
                        row2['Total_Time_Played'] == row['Enemy1TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Enemy1ChampionPlayed']:
                            Enemy1NumR.append(row2['Num_R'])
                            Enemy1NumEX1.append(row2['Num_EX1'])
                            Enemy1NumEX2.append(row2['Num_EX2'])
                            Enemy1NumUltimate.append(row2['Ultimates'])
                            Enemy1DeathOrbs.append(row2['DeathOrbs'])
                            Enemy1HealthShards.append(row2['Health_Shards'])
                            Enemy1EnergyShards.append(row2['Energy_Shards'])
                            Enemy1Division.append(row2['Division'])
                            Enemy1Damage.append(row2['Damage'])
                            Enemy1Protection.append(row2['Protection'])
                            Enemy1ProtectionReceived.append(row2['Protection_Received'])
                            Enemy1Control.append(row2['Control'])
                            Enemy1ControlReceived.append(row2['Control_Received'])
                            Enemy1EnergyUsed.append(row2['Energy_Used'])
                            Enemy1EnergyGained.append(row2['Energy_Gained'])
                            Enemy1Deaths.append(row2['Deaths'])

                        #Ensure same enemy match 2
                        elif row2['Champion'] == row['Enemy2Champion'] and \
                        row2['Total_Time_Played'] == row['Enemy2TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Enemy2ChampionPlayed']:
                            Enemy2NumR.append(row2['Num_R'])
                            Enemy2NumEX1.append(row2['Num_EX1'])
                            Enemy2NumEX2.append(row2['Num_EX2'])
                            Enemy2NumUltimate.append(row2['Ultimates'])
                            Enemy2DeathOrbs.append(row2['DeathOrbs'])
                            Enemy2HealthShards.append(row2['Health_Shards'])
                            Enemy2EnergyShards.append(row2['Energy_Shards'])
                            Enemy2Division.append(row2['Division'])
                            Enemy2Damage.append(row2['Damage'])
                            Enemy2Protection.append(row2['Protection'])
                            Enemy2ProtectionReceived.append(row2['Protection_Received'])
                            Enemy2Control.append(row2['Control'])
                            Enemy2ControlReceived.append(row2['Control_Received'])
                            Enemy2EnergyUsed.append(row2['Energy_Used'])
                            Enemy2EnergyGained.append(row2['Energy_Gained'])
                            Enemy2Deaths.append(row2['Deaths'])

                        else:
                            Enemy3NumR.append(row2['Num_R'])
                            Enemy3NumEX1.append(row2['Num_EX1'])
                            Enemy3NumEX2.append(row2['Num_EX2'])
                            Enemy3NumUltimate.append(row2['Ultimates'])
                            Enemy3DeathOrbs.append(row2['DeathOrbs'])
                            Enemy3HealthShards.append(row2['Health_Shards'])
                            Enemy3EnergyShards.append(row2['Energy_Shards'])
                            Enemy3Division.append(row2['Division'])
                            Enemy3Damage.append(row2['Damage'])
                            Enemy3Protection.append(row2['Protection'])
                            Enemy3ProtectionReceived.append(row2['Protection_Received'])
                            Enemy3Control.append(row2['Control'])
                            Enemy3ControlReceived.append(row2['Control_Received'])
                            Enemy3EnergyUsed.append(row2['Energy_Used'])
                            Enemy3EnergyGained.append(row2['Energy_Gained'])
                            Enemy3Deaths.append(row2['Deaths'])

                #Non-draw rounds
                else:
                    if row2['Name'] != row['Name']:
                        #Same team
                        if row2['Round_Won'] == row['Round_Won']:
                            #Ensure same ally match
                            if row2['Champion'] == row['Ally1Champion']:
                                Ally1NumR.append(row2['Num_R'])
                                Ally1NumEX1.append(row2['Num_EX1'])
                                Ally1NumEX2.append(row2['Num_EX2'])
                                Ally1NumUltimate.append(row2['Ultimates'])
                                Ally1DeathOrbs.append(row2['DeathOrbs'])
                                Ally1HealthShards.append(row2['Health_Shards'])
                                Ally1EnergyShards.append(row2['Energy_Shards'])
                                Ally1Division.append(row2['Division'])
                                Ally1Damage.append(row2['Damage'])
                                Ally1Protection.append(row2['Protection'])
                                Ally1ProtectionReceived.append(row2['Protection_Received'])
                                Ally1Control.append(row2['Control'])
                                Ally1ControlReceived.append(row2['Control_Received'])
                                Ally1EnergyUsed.append(row2['Energy_Used'])
                                Ally1EnergyGained.append(row2['Energy_Gained'])
                                Ally1Deaths.append(row2['Deaths'])
                            else:
                                Ally2NumR.append(row2['Num_R'])
                                Ally2NumEX1.append(row2['Num_EX1'])
                                Ally2NumEX2.append(row2['Num_EX2'])
                                Ally2NumUltimate.append(row2['Ultimates'])
                                Ally2DeathOrbs.append(row2['DeathOrbs'])
                                Ally2HealthShards.append(row2['Health_Shards'])
                                Ally2EnergyShards.append(row2['Energy_Shards'])
                                Ally2Division.append(row2['Division'])
                                Ally2Damage.append(row2['Damage'])
                                Ally2Protection.append(row2['Protection'])
                                Ally2ProtectionReceived.append(row2['Protection_Received'])
                                Ally2Control.append(row2['Control'])
                                Ally2ControlReceived.append(row2['Control_Received'])
                                Ally2EnergyUsed.append(row2['Energy_Used'])
                                Ally2EnergyGained.append(row2['Energy_Gained'])
                                Ally2Deaths.append(row2['Deaths'])
                        #Enemy team
                        else:
                            #Ensure same enemy match
                            if row2['Champion'] == row['Enemy1Champion']:
                                Enemy1NumR.append(row2['Num_R'])
                                Enemy1NumEX1.append(row2['Num_EX1'])
                                Enemy1NumEX2.append(row2['Num_EX2'])
                                Enemy1NumUltimate.append(row2['Ultimates'])
                                Enemy1DeathOrbs.append(row2['DeathOrbs'])
                                Enemy1HealthShards.append(row2['Health_Shards'])
                                Enemy1EnergyShards.append(row2['Energy_Shards'])
                                Enemy1Division.append(row2['Division'])
                                Enemy1Damage.append(row2['Damage'])
                                Enemy1Protection.append(row2['Protection'])
                                Enemy1ProtectionReceived.append(row2['Protection_Received'])
                                Enemy1Control.append(row2['Control'])
                                Enemy1ControlReceived.append(row2['Control_Received'])
                                Enemy1EnergyUsed.append(row2['Energy_Used'])
                                Enemy1EnergyGained.append(row2['Energy_Gained'])
                                Enemy1Deaths.append(row2['Deaths'])
                                
                            elif row2['Champion'] == row['Enemy2Champion']:
                                Enemy2NumR.append(row2['Num_R'])
                                Enemy2NumEX1.append(row2['Num_EX1'])
                                Enemy2NumEX2.append(row2['Num_EX2'])
                                Enemy2NumUltimate.append(row2['Ultimates'])
                                Enemy2DeathOrbs.append(row2['DeathOrbs'])
                                Enemy2HealthShards.append(row2['Health_Shards'])
                                Enemy2EnergyShards.append(row2['Energy_Shards'])
                                Enemy2Division.append(row2['Division'])
                                Enemy2Damage.append(row2['Damage'])
                                Enemy2Protection.append(row2['Protection'])
                                Enemy2ProtectionReceived.append(row2['Protection_Received'])
                                Enemy2Control.append(row2['Control'])
                                Enemy2ControlReceived.append(row2['Control_Received'])
                                Enemy2EnergyUsed.append(row2['Energy_Used'])
                                Enemy2EnergyGained.append(row2['Energy_Gained'])
                                Enemy2Deaths.append(row2['Deaths'])
                                
                            else:
                                Enemy3NumR.append(row2['Num_R'])
                                Enemy3NumEX1.append(row2['Num_EX1'])
                                Enemy3NumEX2.append(row2['Num_EX2'])
                                Enemy3NumUltimate.append(row2['Ultimates'])
                                Enemy3DeathOrbs.append(row2['DeathOrbs'])
                                Enemy3HealthShards.append(row2['Health_Shards'])
                                Enemy3EnergyShards.append(row2['Energy_Shards'])
                                Enemy3Division.append(row2['Division'])
                                Enemy3Damage.append(row2['Damage'])
                                Enemy3Protection.append(row2['Protection'])
                                Enemy3ProtectionReceived.append(row2['Protection_Received'])
                                Enemy3Control.append(row2['Control'])
                                Enemy3ControlReceived.append(row2['Control_Received'])
                                Enemy3EnergyUsed.append(row2['Energy_Used'])
                                Enemy3EnergyGained.append(row2['Energy_Gained'])
                                Enemy3Deaths.append(row2['Deaths'])

            #2v2
            else:
                #Draw rounds
                if sum(round_df['Round_Won']) == 0:
                    if row2['Name'] != row['Name']:
                        #Ensure same team and not enemy team same champion
                        if row2['Champion'] == row['Ally1Champion'] and \
                        row2['Total_Time_Played'] == row['Ally1TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Ally1ChampionPlayed']:
                            Ally1NumR.append(row2['Num_R'])
                            Ally1NumEX1.append(row2['Num_EX1'])
                            Ally1NumEX2.append(row2['Num_EX2'])
                            Ally1NumUltimate.append(row2['Ultimates'])
                            Ally1DeathOrbs.append(row2['DeathOrbs'])
                            Ally1HealthShards.append(row2['Health_Shards'])
                            Ally1EnergyShards.append(row2['Energy_Shards'])
                            Ally1Division.append(row2['Division'])
                            Ally1Damage.append(row2['Damage'])
                            Ally1Protection.append(row2['Protection'])
                            Ally1ProtectionReceived.append(row2['Protection_Received'])
                            Ally1Control.append(row2['Control'])
                            Ally1ControlReceived.append(row2['Control_Received'])
                            Ally1EnergyUsed.append(row2['Energy_Used'])
                            Ally1EnergyGained.append(row2['Energy_Gained'])
                            Ally1Deaths.append(row2['Deaths'])
                            Ally2NumR.append(np.NaN)
                            Ally2NumEX1.append(np.NaN)
                            Ally2NumEX2.append(np.NaN)
                            Ally2NumUltimate.append(np.NaN)
                            Ally2DeathOrbs.append(np.NaN)
                            Ally2HealthShards.append(np.NaN)
                            Ally2EnergyShards.append(np.NaN)
                            Ally2Division.append(np.NaN)
                            Ally2Damage.append(np.NaN)
                            Ally2Protection.append(np.NaN)
                            Ally2ProtectionReceived.append(np.NaN)
                            Ally2Control.append(np.NaN)
                            Ally2ControlReceived.append(np.NaN)
                            Ally2EnergyUsed.append(np.NaN)
                            Ally2EnergyGained.append(np.NaN)
                            Ally2Deaths.append(np.NaN)
                            

                        #Ensure same enemy match
                        elif row2['Champion'] == row['Enemy1Champion'] and \
                        row2['Total_Time_Played'] == row['Enemy1TotalPlayed'] and \
                        row2['Champion_Time_Played'] == row['Enemy1ChampionPlayed']:
                            Enemy1NumR.append(row2['Num_R'])
                            Enemy1NumEX1.append(row2['Num_EX1'])
                            Enemy1NumEX2.append(row2['Num_EX2'])
                            Enemy1NumUltimate.append(row2['Ultimates'])
                            Enemy1DeathOrbs.append(row2['DeathOrbs'])
                            Enemy1HealthShards.append(row2['Health_Shards'])
                            Enemy1EnergyShards.append(row2['Energy_Shards'])
                            Enemy1Division.append(row2['Division'])
                            Enemy1Damage.append(row2['Damage'])
                            Enemy1Protection.append(row2['Protection'])
                            Enemy1ProtectionReceived.append(row2['Protection_Received'])
                            Enemy1Control.append(row2['Control'])
                            Enemy1ControlReceived.append(row2['Control_Received'])
                            Enemy1EnergyUsed.append(row2['Energy_Used'])
                            Enemy1EnergyGained.append(row2['Energy_Gained'])
                            Enemy1Deaths.append(row2['Deaths'])

                        else:
                            Enemy2NumR.append(row2['Num_R'])
                            Enemy2NumEX1.append(row2['Num_EX1'])
                            Enemy2NumEX2.append(row2['Num_EX2'])
                            Enemy2NumUltimate.append(row2['Ultimates'])
                            Enemy2DeathOrbs.append(row2['DeathOrbs'])
                            Enemy2HealthShards.append(row2['Health_Shards'])
                            Enemy2EnergyShards.append(row2['Energy_Shards'])
                            Enemy2Division.append(row2['Division'])
                            Enemy2Damage.append(row2['Damage'])
                            Enemy2Protection.append(row2['Protection'])
                            Enemy2ProtectionReceived.append(row2['Protection_Received'])
                            Enemy2Control.append(row2['Control'])
                            Enemy2ControlReceived.append(row2['Control_Received'])
                            Enemy2EnergyUsed.append(row2['Energy_Used'])
                            Enemy2EnergyGained.append(row2['Energy_Gained'])
                            Enemy2Deaths.append(row2['Deaths'])

                        if not Enemy3:
                            Enemy3NumR.append(np.NaN)
                            Enemy3NumEX1.append(np.NaN)
                            Enemy3NumEX2.append(np.NaN)
                            Enemy3NumUltimate.append(np.NaN)
                            Enemy3DeathOrbs.append(np.NaN)
                            Enemy3HealthShards.append(np.NaN)
                            Enemy3EnergyShards.append(np.NaN)
                            Enemy3Division.append(np.NaN)
                            Enemy3Damage.append(np.NaN)
                            Enemy3Protection.append(np.NaN)
                            Enemy3ProtectionReceived.append(np.NaN)
                            Enemy3Control.append(np.NaN)
                            Enemy3ControlReceived.append(np.NaN)
                            Enemy3EnergyUsed.append(np.NaN)
                            Enemy3EnergyGained.append(np.NaN)
                            Enemy3Deaths.append(np.NaN)
                            Enemy3 = True
                #Non-draw rounds
                else:
                    if row2['Name'] != row['Name']:
                        #Same team
                        if row2['Round_Won'] == row['Round_Won']:
                            if row2['Champion'] == row['Ally1Champion']:
                                Ally1NumR.append(row2['Num_R'])
                                Ally1NumEX1.append(row2['Num_EX1'])
                                Ally1NumEX2.append(row2['Num_EX2'])
                                Ally1NumUltimate.append(row2['Ultimates'])
                                Ally1DeathOrbs.append(row2['DeathOrbs'])
                                Ally1HealthShards.append(row2['Health_Shards'])
                                Ally1EnergyShards.append(row2['Energy_Shards'])
                                Ally1Division.append(row2['Division'])
                                Ally1Damage.append(row2['Damage'])
                                Ally1Protection.append(row2['Protection'])
                                Ally1ProtectionReceived.append(row2['Protection_Received'])
                                Ally1Control.append(row2['Control'])
                                Ally1ControlReceived.append(row2['Control_Received'])
                                Ally1EnergyUsed.append(row2['Energy_Used'])
                                Ally1EnergyGained.append(row2['Energy_Gained'])
                                Ally1Deaths.append(row2['Deaths'])
                                Ally2NumR.append(np.NaN)
                                Ally2NumEX1.append(np.NaN)
                                Ally2NumEX2.append(np.NaN)
                                Ally2NumUltimate.append(np.NaN)
                                Ally2DeathOrbs.append(np.NaN)
                                Ally2HealthShards.append(np.NaN)
                                Ally2EnergyShards.append(np.NaN)
                                Ally2Division.append(np.NaN)
                                Ally2Damage.append(np.NaN)
                                Ally2Protection.append(np.NaN)
                                Ally2ProtectionReceived.append(np.NaN)
                                Ally2Control.append(np.NaN)
                                Ally2ControlReceived.append(np.NaN)
                                Ally2EnergyUsed.append(np.NaN)
                                Ally2EnergyGained.append(np.NaN)
                                Ally2Deaths.append(np.NaN)
                        #Enemy
                        else:
                            #Ensure same enemy match
                            if row2['Champion'] == row['Enemy1Champion']:

                                Enemy1NumR.append(row2['Num_R'])
                                Enemy1NumEX1.append(row2['Num_EX1'])
                                Enemy1NumEX2.append(row2['Num_EX2'])
                                Enemy1NumUltimate.append(row2['Ultimates'])
                                Enemy1DeathOrbs.append(row2['DeathOrbs'])
                                Enemy1HealthShards.append(row2['Health_Shards'])
                                Enemy1EnergyShards.append(row2['Energy_Shards'])
                                Enemy1Division.append(row2['Division'])
                                Enemy1Damage.append(row2['Damage'])
                                Enemy1Protection.append(row2['Protection'])
                                Enemy1ProtectionReceived.append(row2['Protection_Received'])
                                Enemy1Control.append(row2['Control'])
                                Enemy1ControlReceived.append(row2['Control_Received'])
                                Enemy1EnergyUsed.append(row2['Energy_Used'])
                                Enemy1EnergyGained.append(row2['Energy_Gained'])
                                Enemy1Deaths.append(row2['Deaths'])
                                
                            else:
                                Enemy2NumR.append(row2['Num_R'])
                                Enemy2NumEX1.append(row2['Num_EX1'])
                                Enemy2NumEX2.append(row2['Num_EX2'])
                                Enemy2NumUltimate.append(row2['Ultimates'])
                                Enemy2DeathOrbs.append(row2['DeathOrbs'])
                                Enemy2HealthShards.append(row2['Health_Shards'])
                                Enemy2EnergyShards.append(row2['Energy_Shards'])
                                Enemy2Division.append(row2['Division'])
                                Enemy2Damage.append(row2['Damage'])
                                Enemy2Protection.append(row2['Protection'])
                                Enemy2ProtectionReceived.append(row2['Protection_Received'])
                                Enemy2Control.append(row2['Control'])
                                Enemy2ControlReceived.append(row2['Control_Received'])
                                Enemy2EnergyUsed.append(row2['Energy_Used'])
                                Enemy2EnergyGained.append(row2['Energy_Gained'])
                                Enemy2Deaths.append(row2['Deaths'])

                            if not Enemy3:
                                Enemy3NumR.append(np.NaN)
                                Enemy3NumEX1.append(np.NaN)
                                Enemy3NumEX2.append(np.NaN)
                                Enemy3NumUltimate.append(np.NaN)
                                Enemy3DeathOrbs.append(np.NaN)
                                Enemy3HealthShards.append(np.NaN)
                                Enemy3EnergyShards.append(np.NaN)
                                Enemy3Division.append(np.NaN)
                                Enemy3Damage.append(np.NaN)
                                Enemy3Protection.append(np.NaN)
                                Enemy3ProtectionReceived.append(np.NaN)
                                Enemy3Control.append(np.NaN)
                                Enemy3ControlReceived.append(np.NaN)
                                Enemy3EnergyUsed.append(np.NaN)
                                Enemy3EnergyGained.append(np.NaN)
                                Enemy3Deaths.append(np.NaN)
                                Enemy3 = True
                                
     
    #Bad game IDs with inconsistent formatting or impossible scenarios causing inconsistent lengths
    #One scenario involved 4 winning rounds recorded when the maximum is 3 rounds for a ranked game
    if len(df) != len(Ally1NumR) or \
       len(df) != len(Ally2NumR) or \
       len(df) != len(Enemy1NumR) or \
       len(df) != len(Enemy2NumR) or \
       len(df) != len(Enemy3NumR):
        bad_game_ids.append(subset_game_id)
    else:
        FinalAlly1NumR += Ally1NumR
        FinalAlly1NumEX1 += Ally1NumEX1
        FinalAlly1NumEX2 += Ally1NumEX2
        FinalAlly1NumUltimate += Ally1NumUltimate
        FinalAlly1DeathOrbs += Ally1DeathOrbs
        FinalAlly1HealthShards += Ally1HealthShards
        FinalAlly1EnergyShards += Ally1EnergyShards
        FinalAlly1Division += Ally1Division
        FinalAlly1Damage += Ally1Damage
        FinalAlly1Protection += Ally1Protection
        FinalAlly1ProtectionReceived += Ally1ProtectionReceived
        FinalAlly1Control += Ally1Control
        FinalAlly1ControlReceived += Ally1ControlReceived
        FinalAlly1EnergyUsed += Ally1EnergyUsed
        FinalAlly1EnergyGained += Ally1EnergyGained
        FinalAlly1Deaths += Ally1Deaths
        FinalAlly2NumR += Ally2NumR
        FinalAlly2NumEX1 += Ally2NumEX1
        FinalAlly2NumEX2 += Ally2NumEX2
        FinalAlly2NumUltimate += Ally2NumUltimate
        FinalAlly2DeathOrbs += Ally2DeathOrbs
        FinalAlly2HealthShards += Ally2HealthShards
        FinalAlly2EnergyShards += Ally2EnergyShards
        FinalAlly2Division += Ally2Division
        FinalAlly2Damage += Ally1Damage
        FinalAlly2Protection += Ally2Protection
        FinalAlly2ProtectionReceived += Ally2ProtectionReceived
        FinalAlly2Control += Ally2Control
        FinalAlly2ControlReceived += Ally2ControlReceived
        FinalAlly2EnergyUsed += Ally2EnergyUsed
        FinalAlly2EnergyGained += Ally2EnergyGained
        FinalAlly2Deaths += Ally2Deaths
        FinalEnemy1NumR += Enemy1NumR
        FinalEnemy1NumEX1 += Enemy1NumEX1
        FinalEnemy1NumEX2 += Enemy1NumEX2
        FinalEnemy1NumUltimate += Enemy1NumUltimate
        FinalEnemy1DeathOrbs += Enemy1DeathOrbs
        FinalEnemy1HealthShards += Enemy1HealthShards
        FinalEnemy1EnergyShards += Enemy1EnergyShards
        FinalEnemy1Division += Enemy1Division
        FinalEnemy1Damage += Enemy1Damage
        FinalEnemy1Protection += Enemy1Protection
        FinalEnemy1ProtectionReceived += Enemy1ProtectionReceived
        FinalEnemy1Control += Enemy1Control
        FinalEnemy1ControlReceived += Enemy1ControlReceived
        FinalEnemy1EnergyUsed += Enemy1EnergyUsed
        FinalEnemy1EnergyGained += Enemy1EnergyGained
        FinalEnemy1Deaths += Enemy1Deaths
        FinalEnemy2NumR += Enemy2NumR
        FinalEnemy2NumEX1 += Enemy2NumEX1
        FinalEnemy2NumEX2 += Enemy2NumEX2
        FinalEnemy2NumUltimate += Enemy2NumUltimate
        FinalEnemy2DeathOrbs += Enemy2DeathOrbs
        FinalEnemy2HealthShards += Enemy2HealthShards
        FinalEnemy2EnergyShards += Enemy2EnergyShards
        FinalEnemy2Division += Enemy2Division
        FinalEnemy2Damage += Enemy1Damage
        FinalEnemy2Protection += Enemy2Protection
        FinalEnemy2ProtectionReceived += Enemy2ProtectionReceived
        FinalEnemy2Control += Enemy2Control
        FinalEnemy2ControlReceived += Enemy2ControlReceived
        FinalEnemy2EnergyUsed += Enemy2EnergyUsed
        FinalEnemy2EnergyGained += Enemy2EnergyGained
        FinalEnemy2Deaths += Enemy2Deaths
        FinalEnemy3NumR += Enemy3NumR
        FinalEnemy3NumEX1 += Enemy3NumEX1
        FinalEnemy3NumEX2 += Enemy3NumEX2
        FinalEnemy3NumUltimate += Enemy3NumUltimate
        FinalEnemy3DeathOrbs += Enemy3DeathOrbs
        FinalEnemy3HealthShards += Enemy3HealthShards
        FinalEnemy3EnergyShards += Enemy3EnergyShards
        FinalEnemy3Division += Enemy3Division
        FinalEnemy3Damage += Enemy3Damage
        FinalEnemy3Protection += Enemy3Protection
        FinalEnemy3ProtectionReceived += Enemy3ProtectionReceived
        FinalEnemy3Control += Enemy3Control
        FinalEnemy3ControlReceived += Enemy3ControlReceived
        FinalEnemy3EnergyUsed += Enemy3EnergyUsed
        FinalEnemy3EnergyGained += Enemy3EnergyGained
        FinalEnemy3Deaths += Enemy3Deaths

In [19]:
#Set equal to baseline again
df = baseline_df.copy()

#Filter out bad game ids
df = df[~df['Game_ID'].isin(bad_game_ids)].reset_index(drop = True)

In [20]:
#Add new columns of data
df['Ally1NumR'] = FinalAlly1NumR
df['Ally1NumEX1'] = FinalAlly1NumEX1
df['Ally1NumEX2'] = FinalAlly1NumEX2
df['Ally1NumUltimates'] = FinalAlly1NumUltimate
df['Ally1DeathOrbs'] = FinalAlly1DeathOrbs
df['Ally1HealthShards'] = FinalAlly1HealthShards
df['Ally1EnergyShards'] = FinalAlly1EnergyShards
df['Ally1Division'] = FinalAlly1Division
df['Ally1Damage'] = FinalAlly1Damage
df['Ally1Protection'] = FinalAlly1Protection
df['Ally1ProtectionReceived'] = FinalAlly1ProtectionReceived
df['Ally1Control'] = FinalAlly1Control
df['Ally1ControlReceived'] = FinalAlly1ControlReceived
df['Ally1EnergyUsed'] = FinalAlly1EnergyUsed
df['Ally1EnergyGained'] = FinalAlly1EnergyGained
df['Ally1Deaths'] = FinalAlly1Deaths
df['Ally2NumR'] = FinalAlly2NumR
df['Ally2NumEX1'] = FinalAlly2NumEX1
df['Ally2NumEX2'] = FinalAlly2NumEX2
df['Ally2NumUltimates'] = FinalAlly2NumUltimate
df['Ally2DeathOrbs'] = FinalAlly2DeathOrbs
df['Ally2HealthShards'] = FinalAlly2HealthShards
df['Ally2EnergyShards'] = FinalAlly2EnergyShards
df['Ally2Division'] = FinalAlly2Division
df['Ally2Damage'] = FinalAlly2Damage
df['Ally2Protection'] = FinalAlly2Protection
df['Ally2ProtectionReceived'] = FinalAlly2ProtectionReceived
df['Ally2Control'] = FinalAlly2Control
df['Ally2ControlReceived'] = FinalAlly2ControlReceived
df['Ally2EnergyUsed'] = FinalAlly2EnergyUsed
df['Ally2EnergyGained'] = FinalAlly2EnergyGained
df['Ally2Deaths'] = FinalAlly2Deaths
df['Enemy1NumR'] = FinalEnemy1NumR
df['Enemy1NumEX1'] = FinalEnemy1NumEX1
df['Enemy1NumEX2'] = FinalEnemy1NumEX2
df['Enemy1NumUltimates'] = FinalEnemy1NumUltimate
df['Enemy1DeathOrbs'] = FinalEnemy1DeathOrbs
df['Enemy1HealthShards'] = FinalEnemy1HealthShards
df['Enemy1EnergyShards'] = FinalEnemy1EnergyShards
df['Enemy1Division'] = FinalEnemy1Division
df['Enemy1Damage'] = FinalEnemy1Damage
df['Enemy1Protection'] = FinalEnemy1Protection
df['Enemy1ProtectionReceived'] = FinalEnemy1ProtectionReceived
df['Enemy1Control'] = FinalEnemy1Control
df['Enemy1ControlReceived'] = FinalEnemy1ControlReceived
df['Enemy1EnergyUsed'] = FinalEnemy1EnergyUsed
df['Enemy1EnergyGained'] = FinalEnemy1EnergyGained
df['Enemy1Deaths'] = FinalEnemy1Deaths
df['Enemy2NumR'] = FinalEnemy2NumR
df['Enemy2NumEX1'] = FinalEnemy2NumEX1
df['Enemy2NumEX2'] = FinalEnemy2NumEX2
df['Enemy2NumUltimates'] = FinalEnemy2NumUltimate
df['Enemy2DeathOrbs'] = FinalEnemy2DeathOrbs
df['Enemy2HealthShards'] = FinalEnemy2HealthShards
df['Enemy2EnergyShards'] = FinalEnemy2EnergyShards
df['Enemy2Division'] = FinalEnemy2Division
df['Enemy2Damage'] = FinalEnemy2Damage
df['Enemy2Protection'] = FinalEnemy2Protection
df['Enemy2ProtectionReceived'] = FinalEnemy2ProtectionReceived
df['Enemy2Control'] = FinalEnemy2Control
df['Enemy2ControlReceived'] = FinalEnemy2ControlReceived
df['Enemy2EnergyUsed'] = FinalEnemy2EnergyUsed
df['Enemy2EnergyGained'] = FinalEnemy2EnergyGained
df['Enemy2Deaths'] = FinalEnemy2Deaths
df['Enemy3NumR'] = FinalEnemy3NumR
df['Enemy3NumEX1'] = FinalEnemy3NumEX1
df['Enemy3NumEX2'] = FinalEnemy3NumEX2
df['Enemy3NumUltimates'] = FinalEnemy3NumUltimate
df['Enemy3DeathOrbs'] = FinalEnemy3DeathOrbs
df['Enemy3HealthShards'] = FinalEnemy3HealthShards
df['Enemy3EnergyShards'] = FinalEnemy3EnergyShards
df['Enemy3Division'] = FinalEnemy3Division
df['Enemy3Damage'] = FinalEnemy3Damage
df['Enemy3Protection'] = FinalEnemy3Protection
df['Enemy3ProtectionReceived'] = FinalEnemy3ProtectionReceived
df['Enemy3Control'] = FinalEnemy3Control
df['Enemy3ControlReceived'] = FinalEnemy3ControlReceived
df['Enemy3EnergyUsed'] = FinalEnemy3EnergyUsed
df['Enemy3EnergyGained'] = FinalEnemy3EnergyGained
df['Enemy3Deaths'] = FinalEnemy3Deaths

In [21]:
#Create Total Deaths column
df['TotalDeaths'] = df['Deaths'] + df['Ally1Deaths'] + df['Ally2Deaths'].fillna(0) + df['Enemy1Deaths'] + \
                    df['Enemy2Deaths'] + df['Enemy3Deaths'].fillna(0)

#Create League + Division column
df['LeagueDivision'] = df['League'] + ' ' + df['Division'].astype(str)

#Create League + Division columns for allies and enemies
for selection in ['Ally1', 'Ally2', 'Enemy1', 'Enemy2', 'Enemy3']:
    df[selection + 'LeagueDivision'] = df[selection + 'League'] + ' ' + df[selection + 'Division'].astype(str)

#Temporary columns
df['EnemyTotalHealthShards'] = df['Enemy1HealthShards'] + df['Enemy2HealthShards'] + df['Enemy3HealthShards'].fillna(0)
df['EnemyTotalEnergyShards'] = df['Enemy1EnergyShards'] + df['Enemy2EnergyShards'] + df['Enemy3EnergyShards'].fillna(0)
df['TeamDeathOrbs'] = df['DeathOrbs'] + df['Ally1DeathOrbs'] + df['Ally2DeathOrbs'].fillna(0)
df['EnemyDeathOrbs'] = df['Enemy1DeathOrbs'] + df['Enemy2DeathOrbs'] + df['Enemy3DeathOrbs'].fillna(0)
df['TeamTotalUltimates'] = df['Ultimates'] + df['Ally1NumUltimates'] + df['Ally2NumUltimates'].fillna(0)
df['EnemyTotalUltimates'] = df['Enemy1NumUltimates'] + df['Enemy2NumUltimates'] + df['Enemy3NumUltimates'].fillna(0)
df['TeamTotalEX'] = df['Num_EX1'] + df['Num_EX2'] + df['Ally1NumEX1'] + df['Ally1NumEX2'] + \
                    df['Ally2NumEX1'].fillna(0) + df['Ally2NumEX2'].fillna(0)
df['EnemyTotalEX'] = df['Enemy1NumEX1'] + df['Enemy1NumEX2'] + \
                     df['Enemy2NumEX1'] + df['Enemy2NumEX2'] + \
                     df['Enemy3NumEX1'].fillna(0) + df['Enemy3NumEX2'].fillna(0)
df['TeamTotalR'] = df['Num_R'] + df['Ally1NumR'] + df['Ally2NumR'].fillna(0)
df['EnemyTotalR'] = df['Enemy1NumR'] + df['Enemy2NumR'] + df['Enemy3NumR'].fillna(0)

#Defined as ratio of ultimates between all sources of energy usage
df['TeamRatioUltimatesAll'] = df['TeamTotalUltimates'] / \
                           (df['TeamTotalUltimates'] + \
                            df['Num_Energy_Used'] + df['Ally1NumEnergyUsed'] + df['Ally2NumEnergyUsed'].fillna(0))

df['EnemyRatioUltimatesAll'] = df['EnemyTotalUltimates'] / \
                            (df['EnemyTotalUltimates'] + \
                             df['Enemy1NumEnergyUsed'] + df['Enemy2NumEnergyUsed'] + df['Enemy3NumEnergyUsed'].fillna(0))

#Defined as the ratio of R energy usages between R and EX.
df['TeamRatioREX'] = df['TeamTotalR'] / (df['TeamTotalR'] + df['TeamTotalEX'])
df['EnemyRatioREX'] = df['EnemyTotalR'] / (df['EnemyTotalR'] + df['EnemyTotalEX'])


### Generally using Team metrics over individual player metrics because it is not reasonable to track interactions of all the score related variables with the related champion. Without an interaction, individual statistics are meaningless and their effects can't be accurately tracked.

### If there is an effect of the number of ultimates, EX, R then the effect should be present for both sides and so if there is an equal number between the two sides, there should be zero effect. There is the potential for this to be untrue because of interactions between different champions but the dataset is too small to accurately capture this.

### The same holds true for NetOrbs and NetDeathOrbs.

### HealthShards and EnergyShards are a contested resource so it's better to combine into a net difference to reduce collinearity.

### The cutoffs picked for the RatioUltimates/All Energy usage and RatioR/EX are picked what seems to be reasonable based on game knowledge.

In [22]:
#Columns to keep
df['TeamRatioUltimatesAll20'] = df['TeamRatioUltimatesAll'].apply(lambda x: 1 if x >= 0.2 else 0)
df['EnemyRatioUltimatesAll20'] = df['EnemyRatioUltimatesAll'].apply(lambda x: 1 if x >= 0.2 else 0)
df['TeamRatioREX50'] = df['TeamRatioREX'].apply(lambda x: 1 if x >= 0.5 else 0)
df['EnemyRatioREX50'] = df['EnemyRatioREX'].apply(lambda x: 1 if x >= 0.5 else 0)
df['NetHealthShards'] = df['TeamTotalHealthShards']  - df['EnemyTotalHealthShards']
df['NetEnergyShards'] = df['TeamTotalEnergyShards'] - df['EnemyTotalEnergyShards']
df['NetOrbs'] = df['Team_Orbs'] - df['Enemy_Orbs']
df['NetDeathOrbs'] = df['TeamDeathOrbs'] - df['EnemyDeathOrbs']
df['TeamNetUltimates'] = df['TeamTotalUltimates'] - df['EnemyTotalUltimates']
df['TeamNetEX'] = df['TeamTotalEX'] - df['EnemyTotalEX']
df['TeamNetR'] = df['TeamTotalR'] - df['EnemyTotalR']

#Drop useless columns
df = df.drop(['DeathOrbs', 'Ally1DeathOrbs', 'Ally2DeathOrbs', 'Enemy1DeathOrbs', 'Enemy2DeathOrbs',
              'Enemy3DeathOrbs', 'TeamDeathOrbs', 'EnemyDeathOrbs',
              'Ally1NumR', 'Ally1NumEX1', 'Ally1NumEX2', 'Ally1NumUltimates',
              'Ally2NumR', 'Ally2NumEX1', 'Ally2NumEX2', 'Ally2NumUltimates',
              'Enemy1NumR', 'Enemy1NumEX1', 'Enemy1NumEX2', 'Enemy1NumUltimates',
              'Enemy2NumR', 'Enemy2NumEX1', 'Enemy2NumEX2', 'Enemy2NumUltimates',
              'Enemy3NumR', 'Enemy3NumEX1', 'Enemy3NumEX2', 'Enemy3NumUltimates',
              'Ally1HealthShards', 'Ally1EnergyShards', 'Ally2HealthShards', 'Ally2EnergyShards',
              'Enemy1HealthShards', 'Enemy1EnergyShards', 'Enemy2HealthShards', 'Enemy2EnergyShards',
              'Enemy3HealthShards', 'Enemy3EnergyShards', 'EnemyTotalHealthShards', 'EnemyTotalEnergyShards',
              'TeamTotalUltimates', 'EnemyTotalUltimates', 'TeamTotalEX', 'EnemyTotalEX',
              'TeamTotalR', 'EnemyTotalR', 'TeamRatioUltimatesAll', 'EnemyRatioUltimatesAll',
              'TeamRatioREX', 'EnemyRatioREX'], axis = 1)

In [23]:
#Strings not formatted correctly i.e. 1.0 instead of 1
df['Ally2LeagueDivision'] = df['Ally2LeagueDivision'].str[:-2]
df['Enemy3LeagueDivision'] = df['Enemy3LeagueDivision'].str[:-2]

### The following adds rank ordered score and associated metrics to the data. This means that anything with a 1 at the end corresponds to the highest scoring individual on the team or enemy. 2 indicates the next highest and 3 indicates the lowest for 3v3. For 2v2, the 2 indicates the lowest.

### The metrics are good to be ordered this way because otherwise there is no consistency tied to simply Ally1, Ally2, Enemy1, Enemy2, or Enemy3 columns. Any single player could end up in any of those categories across different games, so the effect of any of these columns to predict whether a round is won is not helpful due to high amounts of randomness. By ordering the scores from highest to lowest, the randomness is decreased and a pattern may emerge based on how high the lowest score is and how low the highest score is.

### Net values / difference between the Team and Enemy can be computed as well to help determine what metric has the most impact on winning the round in a logistic regression model (because round won is a 0 1 binary variable).

In [24]:
#Create Rank ordered score and associated metrics
#Anything with a 1 at the end corresponds to the highest scoring individual on the team or enemy
#3 at the end indicates the lowest scoring member.
#Formatting values this way helps control some random effects
TeamDamage1 = []
TeamDamage2 = []
TeamDamage3 = []
EnemyDamage1 = []
EnemyDamage2 = []
EnemyDamage3 = []
TeamProtection1 = []
TeamProtection2 = []
TeamProtection3 = []
EnemyProtection1 = []
EnemyProtection2 = []
EnemyProtection3 = []
TeamProtectionReceived1 = []
TeamProtectionReceived2 = []
TeamProtectionReceived3 = []
EnemyProtectionReceived1 = []
EnemyProtectionReceived2 = []
EnemyProtectionReceived3 = []
TeamControl1 = []
TeamControl2 = []
TeamControl3 = []
EnemyControl1 = []
EnemyControl2 = []
EnemyControl3 = []
TeamControlReceived1 = []
TeamControlReceived2 = []
TeamControlReceived3 = []
EnemyControlReceived1 = []
EnemyControlReceived2 = []
EnemyControlReceived3 = []
TeamEnergyUsed1 = []
TeamEnergyUsed2 = []
TeamEnergyUsed3 = []
EnemyEnergyUsed1 = []
EnemyEnergyUsed2 = []
EnemyEnergyUsed3 = []
TeamEnergyGained1 = []
TeamEnergyGained2 = []
TeamEnergyGained3 = []
EnemyEnergyGained1 = []
EnemyEnergyGained2 = []
EnemyEnergyGained3 = []
TeamPing1 = []
TeamPing2 = []
TeamPing3 = []
EnemyPing1 = []
EnemyPing2 = []
EnemyPing3 = []
TeamDamageReceived1 = []
TeamDamageReceived2 = []
TeamDamageReceived3 = []
EnemyDamageReceived1 = []
EnemyDamageReceived2 = []
EnemyDamageReceived3 = []
TeamTotalScore1 = []
TeamTotalScore2 = []
TeamTotalScore3 = []
EnemyTotalScore1 = []
EnemyTotalScore2 = []
EnemyTotalScore3 = []
TeamTimeAlive1 = []
TeamTimeAlive2 = []
TeamTimeAlive3 = []
EnemyTimeAlive1 = []
EnemyTimeAlive2 = []
EnemyTimeAlive3 = []
TeamScoreDamageRatio1 = []
TeamScoreDamageRatio2 = []
TeamScoreDamageRatio3 = []
EnemyScoreDamageRatio1 = []
EnemyScoreDamageRatio2 = []
EnemyScoreDamageRatio3 = []
TeamScorePerSecond1 = []
TeamScorePerSecond2 = []
TeamScorePerSecond3 = []
EnemyScorePerSecond1 = []
EnemyScorePerSecond2 = []
EnemyScorePerSecond3 = []
TeamTotalTimePlayed1 = []
TeamTotalTimePlayed2 = []
TeamTotalTimePlayed3 = []
EnemyTotalTimePlayed1 = []
EnemyTotalTimePlayed2 = []
EnemyTotalTimePlayed3 = []
TeamChampionTimePlayed1 = []
TeamChampionTimePlayed2 = []
TeamChampionTimePlayed3 = []
EnemyChampionTimePlayed1 = []
EnemyChampionTimePlayed2 = []
EnemyChampionTimePlayed3 = []
TeamLeagueDivision1 = []
TeamLeagueDivision2 = []
TeamLeagueDivision3 = []
EnemyLeagueDivision1 = []
EnemyLeagueDivision2 = []
EnemyLeagueDivision3 = []
TeamChampion1 = []
TeamChampion2 = []
TeamChampion3 = []
EnemyChampion1 = []
EnemyChampion2 = []
EnemyChampion3 = []
TeamRole1 = []
TeamRole2 = []
TeamRole3 = []
EnemyRole1 = []
EnemyRole2 = []
EnemyRole3 = []
TeamRoleDetail1 = []
TeamRoleDetail2 = []
TeamRoleDetail3 = []
EnemyRoleDetail1 = []
EnemyRoleDetail2 = []
EnemyRoleDetail3 = []
for index, row in df.iterrows():
    #3v3
    if row['Server_Type'] == 'QUICK3V3':
        #Person 1 has highest score in round
        if row['Total_Score'] > row['Ally1TotalScore'] and row['Total_Score'] > row['Ally2TotalScore']:
            TeamDamage1.append(row['Damage'])
            TeamTotalScore1.append(row['Total_Score'])
            TeamProtection1.append(row['Protection'])
            TeamProtectionReceived1.append(row['Protection_Received'])
            TeamControl1.append(row['Control'])
            TeamControlReceived1.append(row['Control_Received'])
            TeamEnergyUsed1.append(row['Energy_Used'])
            TeamEnergyGained1.append(row['Energy_Gained'])
            TeamPing1.append(row['Ping'])
            TeamDamageReceived1.append(row['Damage_Received'])
            TeamTimeAlive1.append(row['Time_Alive'])
            TeamScoreDamageRatio1.append(row['ScoreDamageRatio'])
            TeamScorePerSecond1.append(row['ScorePerSecond'])
            TeamTotalTimePlayed1.append(row['Total_Time_Played'])
            TeamChampionTimePlayed1.append(row['Champion_Time_Played'])
            TeamLeagueDivision1.append(row['LeagueDivision'])
            TeamChampion1.append(row['Champion'])
            TeamRole1.append(row['Role'])
            TeamRoleDetail1.append(row['Role_Detail'])
            
            #Ally 1 second highest score
            if row['Ally1TotalScore'] > row['Ally2TotalScore']:
                TeamDamage2.append(row['Ally1Damage'])
                TeamTotalScore2.append(row['Ally1TotalScore'])
                TeamProtection2.append(row['Ally1Protection'])
                TeamProtectionReceived2.append(row['Ally1ProtectionReceived'])
                TeamControl2.append(row['Ally1Control'])
                TeamControlReceived2.append(row['Ally1ControlReceived'])
                TeamEnergyUsed2.append(row['Ally1EnergyUsed'])
                TeamEnergyGained2.append(row['Ally1EnergyGained'])
                TeamPing2.append(row['Ally1Ping'])
                TeamDamageReceived2.append(row['Ally1DamageReceived'])
                TeamTimeAlive2.append(row['Ally1TimeAlive'])
                TeamScoreDamageRatio2.append(row['Ally1ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['Ally1ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Ally1TotalPlayed'])
                TeamChampionTimePlayed2.append(row['Ally1ChampionPlayed'])
                TeamLeagueDivision2.append(row['Ally1LeagueDivision'])
                TeamChampion2.append(row['Ally1Champion'])
                TeamRole2.append(row['Ally1Role'])
                TeamRoleDetail2.append(row['Ally1RoleDetail'])
                
                #Ally 2 lowest score
                TeamDamage3.append(row['Ally2Damage'])
                TeamTotalScore3.append(row['Ally2TotalScore'])
                TeamProtection3.append(row['Ally2Protection'])
                TeamProtectionReceived3.append(row['Ally2ProtectionReceived'])
                TeamControl3.append(row['Ally2Control'])
                TeamControlReceived3.append(row['Ally2ControlReceived'])
                TeamEnergyUsed3.append(row['Ally2EnergyUsed'])
                TeamEnergyGained3.append(row['Ally2EnergyGained'])
                TeamPing3.append(row['Ally2Ping'])
                TeamDamageReceived3.append(row['Ally2DamageReceived'])
                TeamTimeAlive3.append(row['Ally2TimeAlive'])
                TeamScoreDamageRatio3.append(row['Ally2ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['Ally2ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Ally2TotalPlayed'])
                TeamChampionTimePlayed3.append(row['Ally2ChampionPlayed'])
                TeamLeagueDivision3.append(row['Ally2LeagueDivision'])
                TeamChampion3.append(row['Ally2Champion'])
                TeamRole3.append(row['Ally2Role'])
                TeamRoleDetail3.append(row['Ally2RoleDetail'])
                
            #Ally 2 second highest score
            else:
                TeamDamage2.append(row['Ally2Damage'])
                TeamTotalScore2.append(row['Ally2TotalScore'])
                TeamProtection2.append(row['Ally2Protection'])
                TeamProtectionReceived2.append(row['Ally2ProtectionReceived'])
                TeamControl2.append(row['Ally2Control'])
                TeamControlReceived2.append(row['Ally2ControlReceived'])
                TeamEnergyUsed2.append(row['Ally2EnergyUsed'])
                TeamEnergyGained2.append(row['Ally2EnergyGained'])
                TeamPing2.append(row['Ally2Ping'])
                TeamDamageReceived2.append(row['Ally2DamageReceived'])
                TeamTimeAlive2.append(row['Ally2TimeAlive'])
                TeamScoreDamageRatio2.append(row['Ally2ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['Ally2ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Ally2TotalPlayed'])
                TeamChampionTimePlayed2.append(row['Ally2ChampionPlayed'])
                TeamLeagueDivision2.append(row['Ally2LeagueDivision'])
                TeamChampion2.append(row['Ally2Champion'])
                TeamRole2.append(row['Ally2Role'])
                TeamRoleDetail2.append(row['Ally2RoleDetail'])
                
                #Ally 1 lowest score
                TeamDamage3.append(row['Ally1Damage'])
                TeamTotalScore3.append(row['Ally1TotalScore'])
                TeamProtection3.append(row['Ally1Protection'])
                TeamProtectionReceived3.append(row['Ally1ProtectionReceived'])
                TeamControl3.append(row['Ally1Control'])
                TeamControlReceived3.append(row['Ally1ControlReceived'])
                TeamEnergyUsed3.append(row['Ally1EnergyUsed'])
                TeamEnergyGained3.append(row['Ally1EnergyGained'])
                TeamPing3.append(row['Ally1Ping'])
                TeamDamageReceived3.append(row['Ally1DamageReceived'])
                TeamTimeAlive3.append(row['Ally1TimeAlive'])
                TeamScoreDamageRatio3.append(row['Ally1ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['Ally1ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Ally1TotalPlayed'])
                TeamChampionTimePlayed3.append(row['Ally1ChampionPlayed'])
                TeamLeagueDivision3.append(row['Ally1LeagueDivision'])
                TeamChampion3.append(row['Ally1Champion'])
                TeamRole3.append(row['Ally1Role'])
                TeamRoleDetail3.append(row['Ally1RoleDetail'])
                
                
            
        #Ally 1 highest score
        elif row['Ally1TotalScore'] > row['Total_Score'] and row['Ally1TotalScore'] > row['Ally2TotalScore']:
            TeamDamage1.append(row['Ally1Damage'])
            TeamTotalScore1.append(row['Ally1TotalScore'])
            TeamProtection1.append(row['Ally1Protection'])
            TeamProtectionReceived1.append(row['Ally1ProtectionReceived'])
            TeamControl1.append(row['Ally1Control'])
            TeamControlReceived1.append(row['Ally1ControlReceived'])
            TeamEnergyUsed1.append(row['Ally1EnergyUsed'])
            TeamEnergyGained1.append(row['Ally1EnergyGained'])
            TeamPing1.append(row['Ally1Ping'])
            TeamDamageReceived1.append(row['Ally1DamageReceived'])
            TeamTimeAlive1.append(row['Ally1TimeAlive'])
            TeamScoreDamageRatio1.append(row['Ally1ScoreDamageRatio'])
            TeamScorePerSecond1.append(row['Ally1ScorePerSecond'])
            TeamTotalTimePlayed1.append(row['Ally1TotalPlayed'])
            TeamChampionTimePlayed1.append(row['Ally1ChampionPlayed'])
            TeamLeagueDivision1.append(row['Ally1LeagueDivision'])
            TeamChampion1.append(row['Ally1Champion'])
            TeamRole1.append(row['Ally1Role'])
            TeamRoleDetail1.append(row['Ally1RoleDetail'])
            
            #Person 1 second highest score
            if row['Total_Score'] > row['Ally2TotalScore']:
                TeamDamage2.append(row['Damage'])
                TeamTotalScore2.append(row['Total_Score'])
                TeamProtection2.append(row['Protection'])
                TeamProtectionReceived2.append(row['Protection_Received'])
                TeamControl2.append(row['Control'])
                TeamControlReceived2.append(row['Control_Received'])
                TeamEnergyUsed2.append(row['Energy_Used'])
                TeamEnergyGained2.append(row['Energy_Gained'])
                TeamPing2.append(row['Ping'])
                TeamDamageReceived2.append(row['Damage_Received'])
                TeamTimeAlive2.append(row['Time_Alive'])
                TeamScoreDamageRatio2.append(row['ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Total_Time_Played'])
                TeamChampionTimePlayed2.append(row['Champion_Time_Played'])
                TeamLeagueDivision2.append(row['LeagueDivision'])
                TeamChampion2.append(row['Champion'])
                TeamRole2.append(row['Role'])
                TeamRoleDetail2.append(row['Role_Detail'])
                
                #Ally 2 lowest score
                TeamDamage3.append(row['Ally2Damage'])
                TeamTotalScore3.append(row['Ally2TotalScore'])
                TeamProtection3.append(row['Ally2Protection'])
                TeamProtectionReceived3.append(row['Ally2ProtectionReceived'])
                TeamControl3.append(row['Ally2Control'])
                TeamControlReceived3.append(row['Ally2ControlReceived'])
                TeamEnergyUsed3.append(row['Ally2EnergyUsed'])
                TeamEnergyGained3.append(row['Ally2EnergyGained'])
                TeamPing3.append(row['Ally2Ping'])
                TeamDamageReceived3.append(row['Ally2DamageReceived'])
                TeamTimeAlive3.append(row['Ally2TimeAlive'])
                TeamScoreDamageRatio3.append(row['Ally2ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['Ally2ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Ally2TotalPlayed'])
                TeamChampionTimePlayed3.append(row['Ally2ChampionPlayed'])
                TeamLeagueDivision3.append(row['Ally2LeagueDivision'])
                TeamChampion3.append(row['Ally2Champion'])
                TeamRole3.append(row['Ally2Role'])
                TeamRoleDetail3.append(row['Ally2RoleDetail'])
            
            #Ally 2 second highest score
            else:
                TeamDamage2.append(row['Ally2Damage'])
                TeamTotalScore2.append(row['Ally2TotalScore'])
                TeamProtection2.append(row['Ally2Protection'])
                TeamProtectionReceived2.append(row['Ally2ProtectionReceived'])
                TeamControl2.append(row['Ally2Control'])
                TeamControlReceived2.append(row['Ally2ControlReceived'])
                TeamEnergyUsed2.append(row['Ally2EnergyUsed'])
                TeamEnergyGained2.append(row['Ally2EnergyGained'])
                TeamPing2.append(row['Ally2Ping'])
                TeamDamageReceived2.append(row['Ally2DamageReceived'])
                TeamTimeAlive2.append(row['Ally2TimeAlive'])
                TeamScoreDamageRatio2.append(row['Ally2ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['Ally2ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Ally2TotalPlayed'])
                TeamChampionTimePlayed2.append(row['Ally2ChampionPlayed'])
                TeamLeagueDivision2.append(row['Ally2LeagueDivision'])
                TeamChampion2.append(row['Ally2Champion'])
                TeamRole2.append(row['Ally2Role'])
                TeamRoleDetail2.append(row['Ally2RoleDetail'])
                
                #Person 1 lowest score
                TeamDamage3.append(row['Damage'])
                TeamTotalScore3.append(row['Total_Score'])
                TeamProtection3.append(row['Protection'])
                TeamProtectionReceived3.append(row['Protection_Received'])
                TeamControl3.append(row['Control'])
                TeamControlReceived3.append(row['Control_Received'])
                TeamEnergyUsed3.append(row['Energy_Used'])
                TeamEnergyGained3.append(row['Energy_Gained'])
                TeamPing3.append(row['Ping'])
                TeamDamageReceived3.append(row['Damage_Received'])
                TeamTimeAlive3.append(row['Time_Alive'])
                TeamScoreDamageRatio3.append(row['ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Total_Time_Played'])
                TeamChampionTimePlayed3.append(row['Champion_Time_Played'])
                TeamLeagueDivision3.append(row['LeagueDivision'])
                TeamChampion3.append(row['Champion'])
                TeamRole3.append(row['Role'])
                TeamRoleDetail3.append(row['Role_Detail'])
                
        
        #Ally 2 highest score
        else:
            TeamDamage1.append(row['Ally2Damage'])
            TeamTotalScore1.append(row['Ally2TotalScore'])
            TeamProtection1.append(row['Ally2Protection'])
            TeamProtectionReceived1.append(row['Ally2ProtectionReceived'])
            TeamControl1.append(row['Ally2Control'])
            TeamControlReceived1.append(row['Ally2ControlReceived'])
            TeamEnergyUsed1.append(row['Ally2EnergyUsed'])
            TeamEnergyGained1.append(row['Ally2EnergyGained'])
            TeamPing1.append(row['Ally2Ping'])
            TeamDamageReceived1.append(row['Ally2DamageReceived'])
            TeamTimeAlive1.append(row['Ally2TimeAlive'])
            TeamScoreDamageRatio1.append(row['Ally2ScoreDamageRatio'])
            TeamScorePerSecond1.append(row['Ally2ScorePerSecond'])
            TeamTotalTimePlayed1.append(row['Ally2TotalPlayed'])
            TeamChampionTimePlayed1.append(row['Ally2ChampionPlayed'])
            TeamLeagueDivision1.append(row['Ally2LeagueDivision'])
            TeamChampion1.append(row['Ally2Champion'])
            TeamRole1.append(row['Ally2Role'])
            TeamRoleDetail1.append(row['Ally2RoleDetail'])
            
            #First person second highest score
            if row['Total_Score'] > row['Ally1TotalScore']:
                TeamDamage2.append(row['Damage'])
                TeamTotalScore2.append(row['Total_Score'])
                TeamProtection2.append(row['Protection'])
                TeamProtectionReceived2.append(row['Protection_Received'])
                TeamControl2.append(row['Control'])
                TeamControlReceived2.append(row['Control_Received'])
                TeamEnergyUsed2.append(row['Energy_Used'])
                TeamEnergyGained2.append(row['Energy_Gained'])
                TeamPing2.append(row['Ping'])
                TeamDamageReceived2.append(row['Damage_Received'])
                TeamTimeAlive2.append(row['Time_Alive'])
                TeamScoreDamageRatio2.append(row['ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Total_Time_Played'])
                TeamChampionTimePlayed2.append(row['Champion_Time_Played'])
                TeamLeagueDivision2.append(row['LeagueDivision'])
                TeamChampion2.append(row['Champion'])
                TeamRole2.append(row['Role'])
                TeamRoleDetail2.append(row['Role_Detail'])
                
                #Ally 1 lowest score
                TeamDamage3.append(row['Ally1Damage'])
                TeamTotalScore3.append(row['Ally1TotalScore'])
                TeamProtection3.append(row['Ally1Protection'])
                TeamProtectionReceived3.append(row['Ally1ProtectionReceived'])
                TeamControl3.append(row['Ally1Control'])
                TeamControlReceived3.append(row['Ally1ControlReceived'])
                TeamEnergyUsed3.append(row['Ally1EnergyUsed'])
                TeamEnergyGained3.append(row['Ally1EnergyGained'])
                TeamPing3.append(row['Ally1Ping'])
                TeamDamageReceived3.append(row['Ally1DamageReceived'])
                TeamTimeAlive3.append(row['Ally1TimeAlive'])
                TeamScoreDamageRatio3.append(row['Ally1ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['Ally1ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Ally1TotalPlayed'])
                TeamChampionTimePlayed3.append(row['Ally1ChampionPlayed'])
                TeamLeagueDivision3.append(row['Ally1LeagueDivision'])
                TeamChampion3.append(row['Ally1Champion'])
                TeamRole3.append(row['Ally1Role'])
                TeamRoleDetail3.append(row['Ally1RoleDetail'])
                
            #Ally 1 second highest score
            else:
                TeamDamage2.append(row['Ally1Damage'])
                TeamTotalScore2.append(row['Ally1TotalScore'])
                TeamProtection2.append(row['Ally1Protection'])
                TeamProtectionReceived2.append(row['Ally1ProtectionReceived'])
                TeamControl2.append(row['Ally1Control'])
                TeamControlReceived2.append(row['Ally1ControlReceived'])
                TeamEnergyUsed2.append(row['Ally1EnergyUsed'])
                TeamEnergyGained2.append(row['Ally1EnergyGained'])
                TeamPing2.append(row['Ally1Ping'])
                TeamDamageReceived2.append(row['Ally1DamageReceived'])
                TeamTimeAlive2.append(row['Ally1TimeAlive'])
                TeamScoreDamageRatio2.append(row['Ally1ScoreDamageRatio'])
                TeamScorePerSecond2.append(row['Ally1ScorePerSecond'])
                TeamTotalTimePlayed2.append(row['Ally1TotalPlayed'])
                TeamChampionTimePlayed2.append(row['Ally1ChampionPlayed'])
                TeamLeagueDivision2.append(row['Ally1LeagueDivision'])
                TeamChampion2.append(row['Ally1Champion'])
                TeamRole2.append(row['Ally1Role'])
                TeamRoleDetail2.append(row['Ally1RoleDetail'])
                
                #Person 1 lowest score
                TeamDamage3.append(row['Damage'])
                TeamTotalScore3.append(row['Total_Score'])
                TeamProtection3.append(row['Protection'])
                TeamProtectionReceived3.append(row['Protection_Received'])
                TeamControl3.append(row['Control'])
                TeamControlReceived3.append(row['Control_Received'])
                TeamEnergyUsed3.append(row['Energy_Used'])
                TeamEnergyGained3.append(row['Energy_Gained'])
                TeamPing3.append(row['Ping'])
                TeamDamageReceived3.append(row['Damage_Received'])
                TeamTimeAlive3.append(row['Time_Alive'])
                TeamScoreDamageRatio3.append(row['ScoreDamageRatio'])
                TeamScorePerSecond3.append(row['ScorePerSecond'])
                TeamTotalTimePlayed3.append(row['Total_Time_Played'])
                TeamChampionTimePlayed3.append(row['Champion_Time_Played'])
                TeamLeagueDivision3.append(row['LeagueDivision'])
                TeamChampion3.append(row['Champion'])
                TeamRole3.append(row['Role'])
                TeamRoleDetail3.append(row['Role_Detail'])
                
        #Enemy 1 highest score
        if row['Enemy1TotalScore'] > row['Enemy2TotalScore'] and row['Enemy1TotalScore'] > row['Enemy3TotalScore']:
            
            EnemyDamage1.append(row['Enemy1Damage'])
            EnemyTotalScore1.append(row['Enemy1TotalScore'])
            EnemyProtection1.append(row['Enemy1Protection'])
            EnemyProtectionReceived1.append(row['Enemy1ProtectionReceived'])
            EnemyControl1.append(row['Enemy1Control'])
            EnemyControlReceived1.append(row['Enemy1ControlReceived'])
            EnemyEnergyUsed1.append(row['Enemy1EnergyUsed'])
            EnemyEnergyGained1.append(row['Enemy1EnergyGained'])
            EnemyPing1.append(row['Enemy1Ping'])
            EnemyDamageReceived1.append(row['Enemy1DamageReceived'])
            EnemyTimeAlive1.append(row['Enemy1TimeAlive'])
            EnemyScoreDamageRatio1.append(row['Enemy1ScoreDamageRatio'])
            EnemyScorePerSecond1.append(row['Enemy1ScorePerSecond'])
            EnemyTotalTimePlayed1.append(row['Enemy1TotalPlayed'])
            EnemyChampionTimePlayed1.append(row['Enemy1ChampionPlayed'])
            EnemyLeagueDivision1.append(row['Enemy1LeagueDivision'])
            EnemyChampion1.append(row['Enemy1Champion'])
            EnemyRole1.append(row['Enemy1Role'])
            EnemyRoleDetail1.append(row['Enemy1RoleDetail'])
            
            #Enemy 2 second highest score
            if row['Enemy2TotalScore'] > row['Enemy3TotalScore']:
                EnemyDamage2.append(row['Enemy2Damage'])
                EnemyTotalScore2.append(row['Enemy2TotalScore'])
                EnemyProtection2.append(row['Enemy2Protection'])
                EnemyProtectionReceived2.append(row['Enemy2ProtectionReceived'])
                EnemyControl2.append(row['Enemy2Control'])
                EnemyControlReceived2.append(row['Enemy2ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy2EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy2EnergyGained'])
                EnemyPing2.append(row['Enemy2Ping'])
                EnemyDamageReceived2.append(row['Enemy2DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy2TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy2ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy2ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy2TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy2ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy2LeagueDivision'])
                EnemyChampion2.append(row['Enemy2Champion'])
                EnemyRole2.append(row['Enemy2Role'])
                EnemyRoleDetail2.append(row['Enemy2RoleDetail'])
                
                #Enemy 3 lowest score
                EnemyDamage3.append(row['Enemy3Damage'])
                EnemyTotalScore3.append(row['Enemy3TotalScore'])
                EnemyProtection3.append(row['Enemy3Protection'])
                EnemyProtectionReceived3.append(row['Enemy3ProtectionReceived'])
                EnemyControl3.append(row['Enemy3Control'])
                EnemyControlReceived3.append(row['Enemy3ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy3EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy3EnergyGained'])
                EnemyPing3.append(row['Enemy3Ping'])
                EnemyDamageReceived3.append(row['Enemy3DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy3TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy3ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy3ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy3TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy3ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy3LeagueDivision'])
                EnemyChampion3.append(row['Enemy3Champion'])
                EnemyRole3.append(row['Enemy3Role'])
                EnemyRoleDetail3.append(row['Enemy3RoleDetail'])
                
            #Enemy 3 second highest score
            else:
                EnemyDamage2.append(row['Enemy3Damage'])
                EnemyTotalScore2.append(row['Enemy3TotalScore'])
                EnemyProtection2.append(row['Enemy3Protection'])
                EnemyProtectionReceived2.append(row['Enemy3ProtectionReceived'])
                EnemyControl2.append(row['Enemy3Control'])
                EnemyControlReceived2.append(row['Enemy3ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy3EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy3EnergyGained'])
                EnemyPing2.append(row['Enemy3Ping'])
                EnemyDamageReceived2.append(row['Enemy3DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy3TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy3ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy3ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy3TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy3ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy3LeagueDivision'])
                EnemyChampion2.append(row['Enemy3Champion'])
                EnemyRole2.append(row['Enemy3Role'])
                EnemyRoleDetail2.append(row['Enemy3RoleDetail'])
                
                #Enemy 2 lowest score
                EnemyDamage3.append(row['Enemy2Damage'])
                EnemyTotalScore3.append(row['Enemy2TotalScore'])
                EnemyProtection3.append(row['Enemy2Protection'])
                EnemyProtectionReceived3.append(row['Enemy2ProtectionReceived'])
                EnemyControl3.append(row['Enemy2Control'])
                EnemyControlReceived3.append(row['Enemy2ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy2EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy2EnergyGained'])
                EnemyPing3.append(row['Enemy2Ping'])
                EnemyDamageReceived3.append(row['Enemy2DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy2TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy2ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy2ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy2TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy2ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy2LeagueDivision'])
                EnemyChampion3.append(row['Enemy2Champion'])
                EnemyRole3.append(row['Enemy2Role'])
                EnemyRoleDetail3.append(row['Enemy2RoleDetail'])
                
        #Enemy 2 highest score
        elif row['Enemy2TotalScore'] > row['Enemy1TotalScore'] and row['Enemy2TotalScore'] > row['Enemy3TotalScore']:
            EnemyDamage1.append(row['Enemy2Damage'])
            EnemyTotalScore1.append(row['Enemy2TotalScore'])
            EnemyProtection1.append(row['Enemy2Protection'])
            EnemyProtectionReceived1.append(row['Enemy2ProtectionReceived'])
            EnemyControl1.append(row['Enemy2Control'])
            EnemyControlReceived1.append(row['Enemy2ControlReceived'])
            EnemyEnergyUsed1.append(row['Enemy2EnergyUsed'])
            EnemyEnergyGained1.append(row['Enemy2EnergyGained'])
            EnemyPing1.append(row['Enemy2Ping'])
            EnemyDamageReceived1.append(row['Enemy2DamageReceived'])
            EnemyTimeAlive1.append(row['Enemy2TimeAlive'])
            EnemyScoreDamageRatio1.append(row['Enemy2ScoreDamageRatio'])
            EnemyScorePerSecond1.append(row['Enemy2ScorePerSecond'])
            EnemyTotalTimePlayed1.append(row['Enemy2TotalPlayed'])
            EnemyChampionTimePlayed1.append(row['Enemy2ChampionPlayed'])
            EnemyLeagueDivision1.append(row['Enemy2LeagueDivision'])
            EnemyChampion1.append(row['Enemy2Champion'])
            EnemyRole1.append(row['Enemy2Role'])
            EnemyRoleDetail1.append(row['Enemy2RoleDetail'])
            
            #Enemy 1 second highest score
            if row['Enemy1TotalScore'] > row['Enemy3TotalScore']:
                EnemyDamage2.append(row['Enemy1Damage'])
                EnemyTotalScore2.append(row['Enemy1TotalScore'])
                EnemyProtection2.append(row['Enemy1Protection'])
                EnemyProtectionReceived2.append(row['Enemy1ProtectionReceived'])
                EnemyControl2.append(row['Enemy1Control'])
                EnemyControlReceived2.append(row['Enemy1ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy1EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy1EnergyGained'])
                EnemyPing2.append(row['Enemy1Ping'])
                EnemyDamageReceived2.append(row['Enemy1DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy1TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy1ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy1ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy1TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy1ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy1LeagueDivision'])
                EnemyChampion2.append(row['Enemy1Champion'])
                EnemyRole2.append(row['Enemy1Role'])
                EnemyRoleDetail2.append(row['Enemy1RoleDetail'])
                
                #Enemy 3 lowest score
                EnemyDamage3.append(row['Enemy3Damage'])
                EnemyTotalScore3.append(row['Enemy3TotalScore'])
                EnemyProtection3.append(row['Enemy3Protection'])
                EnemyProtectionReceived3.append(row['Enemy3ProtectionReceived'])
                EnemyControl3.append(row['Enemy3Control'])
                EnemyControlReceived3.append(row['Enemy3ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy3EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy3EnergyGained'])
                EnemyPing3.append(row['Enemy3Ping'])
                EnemyDamageReceived3.append(row['Enemy3DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy3TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy3ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy3ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy3TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy3ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy3LeagueDivision'])
                EnemyChampion3.append(row['Enemy3Champion'])
                EnemyRole3.append(row['Enemy3Role'])
                EnemyRoleDetail3.append(row['Enemy3RoleDetail'])
            
            #Enemy 3 second highest score
            else:
                EnemyDamage2.append(row['Enemy3Damage'])
                EnemyTotalScore2.append(row['Enemy3TotalScore'])
                EnemyProtection2.append(row['Enemy3Protection'])
                EnemyProtectionReceived2.append(row['Enemy3ProtectionReceived'])
                EnemyControl2.append(row['Enemy3Control'])
                EnemyControlReceived2.append(row['Enemy3ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy3EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy3EnergyGained'])
                EnemyPing2.append(row['Enemy3Ping'])
                EnemyDamageReceived2.append(row['Enemy3DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy3TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy3ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy3ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy3TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy3ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy3LeagueDivision'])
                EnemyChampion2.append(row['Enemy3Champion'])
                EnemyRole2.append(row['Enemy3Role'])
                EnemyRoleDetail2.append(row['Enemy3RoleDetail'])
                
                #Enemy 1 lowest score
                EnemyDamage3.append(row['Enemy1Damage'])
                EnemyTotalScore3.append(row['Enemy1TotalScore'])
                EnemyProtection3.append(row['Enemy1Protection'])
                EnemyProtectionReceived3.append(row['Enemy1ProtectionReceived'])
                EnemyControl3.append(row['Enemy1Control'])
                EnemyControlReceived3.append(row['Enemy1ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy1EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy1EnergyGained'])
                EnemyPing3.append(row['Enemy1Ping'])
                EnemyDamageReceived3.append(row['Enemy1DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy1TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy1ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy1ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy1TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy1ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy1LeagueDivision'])
                EnemyChampion3.append(row['Enemy1Champion'])
                EnemyRole3.append(row['Enemy1Role'])
                EnemyRoleDetail3.append(row['Enemy1RoleDetail'])

        #Enemy 3 highest score
        else:
            EnemyDamage1.append(row['Enemy3Damage'])
            EnemyTotalScore1.append(row['Enemy3TotalScore'])
            EnemyProtection1.append(row['Enemy3Protection'])
            EnemyProtectionReceived1.append(row['Enemy3ProtectionReceived'])
            EnemyControl1.append(row['Enemy3Control'])
            EnemyControlReceived1.append(row['Enemy3ControlReceived'])
            EnemyEnergyUsed1.append(row['Enemy3EnergyUsed'])
            EnemyEnergyGained1.append(row['Enemy3EnergyGained'])
            EnemyPing1.append(row['Enemy3Ping'])
            EnemyDamageReceived1.append(row['Enemy3DamageReceived'])
            EnemyTimeAlive1.append(row['Enemy3TimeAlive'])
            EnemyScoreDamageRatio1.append(row['Enemy3ScoreDamageRatio'])
            EnemyScorePerSecond1.append(row['Enemy3ScorePerSecond'])
            EnemyTotalTimePlayed1.append(row['Enemy3TotalPlayed'])
            EnemyChampionTimePlayed1.append(row['Enemy3ChampionPlayed'])
            EnemyLeagueDivision1.append(row['Enemy3LeagueDivision'])
            EnemyChampion1.append(row['Enemy3Champion'])
            EnemyRole1.append(row['Enemy3Role'])
            EnemyRoleDetail1.append(row['Enemy3RoleDetail'])
            
            #Enemy 1 second highest score
            if row['Enemy1TotalScore'] > row['Enemy2TotalScore']:
                EnemyDamage2.append(row['Enemy1Damage'])
                EnemyTotalScore2.append(row['Enemy1TotalScore'])
                EnemyProtection2.append(row['Enemy1Protection'])
                EnemyProtectionReceived2.append(row['Enemy1ProtectionReceived'])
                EnemyControl2.append(row['Enemy1Control'])
                EnemyControlReceived2.append(row['Enemy1ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy1EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy1EnergyGained'])
                EnemyPing2.append(row['Enemy1Ping'])
                EnemyDamageReceived2.append(row['Enemy1DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy1TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy1ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy1ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy1TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy1ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy1LeagueDivision'])
                EnemyChampion2.append(row['Enemy1Champion'])
                EnemyRole2.append(row['Enemy1Role'])
                EnemyRoleDetail2.append(row['Enemy1RoleDetail'])
                
                #Enemy 2 lowest score
                EnemyDamage3.append(row['Enemy2Damage'])
                EnemyTotalScore3.append(row['Enemy2TotalScore'])
                EnemyProtection3.append(row['Enemy2Protection'])
                EnemyProtectionReceived3.append(row['Enemy2ProtectionReceived'])
                EnemyControl3.append(row['Enemy2Control'])
                EnemyControlReceived3.append(row['Enemy2ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy2EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy2EnergyGained'])
                EnemyPing3.append(row['Enemy2Ping'])
                EnemyDamageReceived3.append(row['Enemy2DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy2TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy2ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy2ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy2TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy2ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy2LeagueDivision'])
                EnemyChampion3.append(row['Enemy2Champion'])
                EnemyRole3.append(row['Enemy2Role'])
                EnemyRoleDetail3.append(row['Enemy2RoleDetail'])
                
            #Enemy 2 second highest score
            else:
                EnemyDamage2.append(row['Enemy2Damage'])
                EnemyTotalScore2.append(row['Enemy2TotalScore'])
                EnemyProtection2.append(row['Enemy2Protection'])
                EnemyProtectionReceived2.append(row['Enemy2ProtectionReceived'])
                EnemyControl2.append(row['Enemy2Control'])
                EnemyControlReceived2.append(row['Enemy2ControlReceived'])
                EnemyEnergyUsed2.append(row['Enemy2EnergyUsed'])
                EnemyEnergyGained2.append(row['Enemy2EnergyGained'])
                EnemyPing2.append(row['Enemy2Ping'])
                EnemyDamageReceived2.append(row['Enemy2DamageReceived'])
                EnemyTimeAlive2.append(row['Enemy2TimeAlive'])
                EnemyScoreDamageRatio2.append(row['Enemy2ScoreDamageRatio'])
                EnemyScorePerSecond2.append(row['Enemy2ScorePerSecond'])
                EnemyTotalTimePlayed2.append(row['Enemy2TotalPlayed'])
                EnemyChampionTimePlayed2.append(row['Enemy2ChampionPlayed'])
                EnemyLeagueDivision2.append(row['Enemy2LeagueDivision'])
                EnemyChampion2.append(row['Enemy2Champion'])
                EnemyRole2.append(row['Enemy2Role'])
                EnemyRoleDetail2.append(row['Enemy2RoleDetail'])
                
                #Enemy 1 lowest score
                EnemyDamage3.append(row['Enemy1Damage'])
                EnemyTotalScore3.append(row['Enemy1TotalScore'])
                EnemyProtection3.append(row['Enemy1Protection'])
                EnemyProtectionReceived3.append(row['Enemy1ProtectionReceived'])
                EnemyControl3.append(row['Enemy1Control'])
                EnemyControlReceived3.append(row['Enemy1ControlReceived'])
                EnemyEnergyUsed3.append(row['Enemy1EnergyUsed'])
                EnemyEnergyGained3.append(row['Enemy1EnergyGained'])
                EnemyPing3.append(row['Enemy1Ping'])
                EnemyDamageReceived3.append(row['Enemy1DamageReceived'])
                EnemyTimeAlive3.append(row['Enemy1TimeAlive'])
                EnemyScoreDamageRatio3.append(row['Enemy1ScoreDamageRatio'])
                EnemyScorePerSecond3.append(row['Enemy1ScorePerSecond'])
                EnemyTotalTimePlayed3.append(row['Enemy1TotalPlayed'])
                EnemyChampionTimePlayed3.append(row['Enemy1ChampionPlayed'])
                EnemyLeagueDivision3.append(row['Enemy1LeagueDivision'])
                EnemyChampion3.append(row['Enemy1Champion'])
                EnemyRole3.append(row['Enemy1Role'])
                EnemyRoleDetail3.append(row['Enemy1RoleDetail'])
                
    #2v2            
    else:
        
        #Person 1 has highest score in round
        if row['Total_Score'] > row['Ally1TotalScore']:
            TeamDamage1.append(row['Damage'])
            TeamTotalScore1.append(row['Total_Score'])
            TeamProtection1.append(row['Protection'])
            TeamProtectionReceived1.append(row['Protection_Received'])
            TeamControl1.append(row['Control'])
            TeamControlReceived1.append(row['Control_Received'])
            TeamEnergyUsed1.append(row['Energy_Used'])
            TeamEnergyGained1.append(row['Energy_Gained'])
            TeamPing1.append(row['Ping'])
            TeamDamageReceived1.append(row['Damage_Received'])
            TeamTimeAlive1.append(row['Time_Alive'])
            TeamScoreDamageRatio1.append(row['ScoreDamageRatio'])
            TeamScorePerSecond1.append(row['ScorePerSecond'])
            TeamTotalTimePlayed1.append(row['Total_Time_Played'])
            TeamChampionTimePlayed1.append(row['Champion_Time_Played'])
            TeamLeagueDivision1.append(row['LeagueDivision'])
            TeamChampion1.append(row['Champion'])
            TeamRole1.append(row['Role'])
            TeamRoleDetail1.append(row['Role_Detail'])
            
            #Ally lowest score
            TeamDamage2.append(row['Ally1Damage'])
            TeamTotalScore2.append(row['Ally1TotalScore'])
            TeamProtection2.append(row['Ally1Protection'])
            TeamProtectionReceived2.append(row['Ally1ProtectionReceived'])
            TeamControl2.append(row['Ally1Control'])
            TeamControlReceived2.append(row['Ally1ControlReceived'])
            TeamEnergyUsed2.append(row['Ally1EnergyUsed'])
            TeamEnergyGained2.append(row['Ally1EnergyGained'])
            TeamPing2.append(row['Ally1Ping'])
            TeamDamageReceived2.append(row['Ally1DamageReceived'])
            TeamTimeAlive2.append(row['Ally1TimeAlive'])
            TeamScoreDamageRatio2.append(row['Ally1ScoreDamageRatio'])
            TeamScorePerSecond2.append(row['Ally1ScorePerSecond'])
            TeamTotalTimePlayed2.append(row['Ally1TotalPlayed'])
            TeamChampionTimePlayed2.append(row['Ally1ChampionPlayed'])
            TeamLeagueDivision2.append(row['Ally1LeagueDivision'])
            TeamChampion2.append(row['Ally1Champion'])
            TeamRole2.append(row['Ally1Role'])
            TeamRoleDetail2.append(row['Ally1RoleDetail'])
            
        #Ally 1 highest score
        else:
            
            TeamDamage1.append(row['Ally1Damage'])
            TeamTotalScore1.append(row['Ally1TotalScore'])
            TeamProtection1.append(row['Ally1Protection'])
            TeamProtectionReceived1.append(row['Ally1ProtectionReceived'])
            TeamControl1.append(row['Ally1Control'])
            TeamControlReceived1.append(row['Ally1ControlReceived'])
            TeamEnergyUsed1.append(row['Ally1EnergyUsed'])
            TeamEnergyGained1.append(row['Ally1EnergyGained'])
            TeamPing1.append(row['Ally1Ping'])
            TeamDamageReceived1.append(row['Ally1DamageReceived'])
            TeamTimeAlive1.append(row['Ally1TimeAlive'])
            TeamScoreDamageRatio1.append(row['Ally1ScoreDamageRatio'])
            TeamScorePerSecond1.append(row['Ally1ScorePerSecond'])
            TeamTotalTimePlayed1.append(row['Ally1TotalPlayed'])
            TeamChampionTimePlayed1.append(row['Ally1ChampionPlayed'])
            TeamLeagueDivision1.append(row['Ally1LeagueDivision'])
            TeamChampion1.append(row['Ally1Champion'])
            TeamRole1.append(row['Ally1Role'])
            TeamRoleDetail1.append(row['Ally1RoleDetail'])
            
            #Person 1 lowest score
            TeamDamage2.append(row['Damage'])
            TeamTotalScore2.append(row['Total_Score'])
            TeamProtection2.append(row['Protection'])
            TeamProtectionReceived2.append(row['Protection_Received'])
            TeamControl2.append(row['Control'])
            TeamControlReceived2.append(row['Control_Received'])
            TeamEnergyUsed2.append(row['Energy_Used'])
            TeamEnergyGained2.append(row['Energy_Gained'])
            TeamPing2.append(row['Ping'])
            TeamDamageReceived2.append(row['Damage_Received'])
            TeamTimeAlive2.append(row['Time_Alive'])
            TeamScoreDamageRatio2.append(row['ScoreDamageRatio'])
            TeamScorePerSecond2.append(row['ScorePerSecond'])
            TeamTotalTimePlayed2.append(row['Total_Time_Played'])
            TeamChampionTimePlayed2.append(row['Champion_Time_Played'])
            TeamLeagueDivision2.append(row['LeagueDivision'])
            TeamChampion2.append(row['Champion'])
            TeamRole2.append(row['Role'])
            TeamRoleDetail2.append(row['Role_Detail'])
            
                
        #Ally 2 doesn't exist
        TeamDamage3.append(np.NaN)
        TeamTotalScore3.append(np.NaN)
        TeamProtection3.append(np.NaN)
        TeamProtectionReceived3.append(np.NaN)
        TeamControl3.append(np.NaN)
        TeamControlReceived3.append(np.NaN)
        TeamEnergyUsed3.append(np.NaN)
        TeamEnergyGained3.append(np.NaN)
        TeamPing3.append(np.NaN)
        TeamDamageReceived3.append(np.NaN)
        TeamTimeAlive3.append(np.NaN)
        TeamScoreDamageRatio3.append(np.NaN)
        TeamScorePerSecond3.append(np.NaN)
        TeamTotalTimePlayed3.append(np.NaN)
        TeamChampionTimePlayed3.append(np.NaN)
        TeamLeagueDivision3.append(np.NaN)
        TeamChampion3.append(np.NaN)
        TeamRole3.append(np.NaN)
        TeamRoleDetail3.append(np.NaN)
        
        #Enemy 1 highest score
        if row['Enemy1TotalScore'] > row['Enemy2TotalScore']:
            
            EnemyDamage1.append(row['Enemy1Damage'])
            EnemyTotalScore1.append(row['Enemy1TotalScore'])
            EnemyProtection1.append(row['Enemy1Protection'])
            EnemyProtectionReceived1.append(row['Enemy1ProtectionReceived'])
            EnemyControl1.append(row['Enemy1Control'])
            EnemyControlReceived1.append(row['Enemy1ControlReceived'])
            EnemyEnergyUsed1.append(row['Enemy1EnergyUsed'])
            EnemyEnergyGained1.append(row['Enemy1EnergyGained'])
            EnemyPing1.append(row['Enemy1Ping'])
            EnemyDamageReceived1.append(row['Enemy1DamageReceived'])
            EnemyTimeAlive1.append(row['Enemy1TimeAlive'])
            EnemyScoreDamageRatio1.append(row['Enemy1ScoreDamageRatio'])
            EnemyScorePerSecond1.append(row['Enemy1ScorePerSecond'])
            EnemyTotalTimePlayed1.append(row['Enemy1TotalPlayed'])
            EnemyChampionTimePlayed1.append(row['Enemy1ChampionPlayed'])
            EnemyLeagueDivision1.append(row['Enemy1LeagueDivision'])
            EnemyChampion1.append(row['Enemy1Champion'])
            EnemyRole1.append(row['Enemy1Role'])
            EnemyRoleDetail1.append(row['Enemy1RoleDetail'])
            
            #Enemy 2 lowest score
            EnemyDamage2.append(row['Enemy2Damage'])
            EnemyTotalScore2.append(row['Enemy2TotalScore'])
            EnemyProtection2.append(row['Enemy2Protection'])
            EnemyProtectionReceived2.append(row['Enemy2ProtectionReceived'])
            EnemyControl2.append(row['Enemy2Control'])
            EnemyControlReceived2.append(row['Enemy2ControlReceived'])
            EnemyEnergyUsed2.append(row['Enemy2EnergyUsed'])
            EnemyEnergyGained2.append(row['Enemy2EnergyGained'])
            EnemyPing2.append(row['Enemy2Ping'])
            EnemyDamageReceived2.append(row['Enemy2DamageReceived'])
            EnemyTimeAlive2.append(row['Enemy2TimeAlive'])
            EnemyScoreDamageRatio2.append(row['Enemy2ScoreDamageRatio'])
            EnemyScorePerSecond2.append(row['Enemy2ScorePerSecond'])
            EnemyTotalTimePlayed2.append(row['Enemy2TotalPlayed'])
            EnemyChampionTimePlayed2.append(row['Enemy2ChampionPlayed'])
            EnemyLeagueDivision2.append(row['Enemy2LeagueDivision'])
            EnemyChampion2.append(row['Enemy2Champion'])
            EnemyRole2.append(row['Enemy2Role'])
            EnemyRoleDetail2.append(row['Enemy2RoleDetail'])
        
        #Enemy 2 highest score
        else:
            EnemyDamage1.append(row['Enemy2Damage'])
            EnemyTotalScore1.append(row['Enemy2TotalScore'])
            EnemyProtection1.append(row['Enemy2Protection'])
            EnemyProtectionReceived1.append(row['Enemy2ProtectionReceived'])
            EnemyControl1.append(row['Enemy2Control'])
            EnemyControlReceived1.append(row['Enemy2ControlReceived'])
            EnemyEnergyUsed1.append(row['Enemy2EnergyUsed'])
            EnemyEnergyGained1.append(row['Enemy2EnergyGained'])
            EnemyPing1.append(row['Enemy2Ping'])
            EnemyDamageReceived1.append(row['Enemy2DamageReceived'])
            EnemyTimeAlive1.append(row['Enemy2TimeAlive'])
            EnemyScoreDamageRatio1.append(row['Enemy2ScoreDamageRatio'])
            EnemyScorePerSecond1.append(row['Enemy2ScorePerSecond'])
            EnemyTotalTimePlayed1.append(row['Enemy2TotalPlayed'])
            EnemyChampionTimePlayed1.append(row['Enemy2ChampionPlayed'])
            EnemyLeagueDivision1.append(row['Enemy2LeagueDivision'])
            EnemyChampion1.append(row['Enemy2Champion'])
            EnemyRole1.append(row['Enemy2Role'])
            EnemyRoleDetail1.append(row['Enemy2RoleDetail'])
            
            #Enemy 1 lowest score
            EnemyDamage2.append(row['Enemy1Damage'])
            EnemyTotalScore2.append(row['Enemy1TotalScore'])
            EnemyProtection2.append(row['Enemy1Protection'])
            EnemyProtectionReceived2.append(row['Enemy1ProtectionReceived'])
            EnemyControl2.append(row['Enemy1Control'])
            EnemyControlReceived2.append(row['Enemy1ControlReceived'])
            EnemyEnergyUsed2.append(row['Enemy1EnergyUsed'])
            EnemyEnergyGained2.append(row['Enemy1EnergyGained'])
            EnemyPing2.append(row['Enemy1Ping'])
            EnemyDamageReceived2.append(row['Enemy1DamageReceived'])
            EnemyTimeAlive2.append(row['Enemy1TimeAlive'])
            EnemyScoreDamageRatio2.append(row['Enemy1ScoreDamageRatio'])
            EnemyScorePerSecond2.append(row['Enemy1ScorePerSecond'])
            EnemyTotalTimePlayed2.append(row['Enemy1TotalPlayed'])
            EnemyChampionTimePlayed2.append(row['Enemy1ChampionPlayed'])
            EnemyLeagueDivision2.append(row['Enemy1LeagueDivision'])
            EnemyChampion2.append(row['Enemy1Champion'])
            EnemyRole2.append(row['Enemy1Role'])
            EnemyRoleDetail2.append(row['Enemy1RoleDetail'])
                
        #Enemy 3 doesn't exist
        EnemyDamage3.append(np.NaN)
        EnemyTotalScore3.append(np.NaN)
        EnemyProtection3.append(np.NaN)
        EnemyProtectionReceived3.append(np.NaN)
        EnemyControl3.append(np.NaN)
        EnemyControlReceived3.append(np.NaN)
        EnemyEnergyUsed3.append(np.NaN)
        EnemyEnergyGained3.append(np.NaN)
        EnemyPing3.append(np.NaN)
        EnemyDamageReceived3.append(np.NaN)
        EnemyTimeAlive3.append(np.NaN)
        EnemyScoreDamageRatio3.append(np.NaN)
        EnemyScorePerSecond3.append(np.NaN)
        EnemyTotalTimePlayed3.append(np.NaN)
        EnemyChampionTimePlayed3.append(np.NaN)
        EnemyLeagueDivision3.append(np.NaN)
        EnemyChampion3.append(np.NaN)
        EnemyRole3.append(np.NaN)
        EnemyRoleDetail3.append(np.NaN)
 

In [25]:
#Add columns
df['TeamDamage1'] = TeamDamage1
df['TeamDamage2'] = TeamDamage2
df['TeamDamage3'] = TeamDamage3
df['EnemyDamage1'] = EnemyDamage1
df['EnemyDamage2'] = EnemyDamage2
df['EnemyDamage3'] = EnemyDamage3
df['TeamProtection1'] = TeamProtection1
df['TeamProtection2'] = TeamProtection2
df['TeamProtection3'] = TeamProtection3
df['EnemyProtection1'] = EnemyProtection1
df['EnemyProtection2'] = EnemyProtection2
df['EnemyProtection3'] = EnemyProtection3
df['TeamProtectionReceived1'] = TeamProtectionReceived1
df['TeamProtectionReceived2'] = TeamProtectionReceived2
df['TeamProtectionReceived3'] = TeamProtectionReceived3
df['EnemyProtectionReceived1'] = EnemyProtectionReceived1
df['EnemyProtectionReceived2'] = EnemyProtectionReceived2
df['EnemyProtectionReceived3'] = EnemyProtectionReceived3
df['TeamControl1'] = TeamControl1
df['TeamControl2'] = TeamControl2
df['TeamControl3'] = TeamControl3
df['EnemyControl1'] = EnemyControl1
df['EnemyControl2'] = EnemyControl2
df['EnemyControl3'] = EnemyControl3
df['TeamControlReceived1'] = TeamControlReceived1
df['TeamControlReceived2'] = TeamControlReceived2
df['TeamControlReceived3'] = TeamControlReceived3
df['EnemyControlReceived1'] = EnemyControlReceived1
df['EnemyControlReceived2'] = EnemyControlReceived2
df['EnemyControlReceived3'] = EnemyControlReceived3
df['TeamEnergyUsed1'] = TeamEnergyUsed1
df['TeamEnergyUsed2'] = TeamEnergyUsed2
df['TeamEnergyUsed3'] = TeamEnergyUsed3
df['EnemyEnergyUsed1'] = EnemyEnergyUsed1
df['EnemyEnergyUsed2'] = EnemyEnergyUsed2
df['EnemyEnergyUsed3'] = EnemyEnergyUsed3
df['TeamEnergyGained1'] = TeamEnergyGained1
df['TeamEnergyGained2'] = TeamEnergyGained2
df['TeamEnergyGained3'] = TeamEnergyGained3
df['EnemyEnergyGained1'] = EnemyEnergyGained1
df['EnemyEnergyGained2'] = EnemyEnergyGained2
df['EnemyEnergyGained3'] = EnemyEnergyGained3
df['TeamPing1'] = TeamPing1
df['TeamPing2'] = TeamPing2
df['TeamPing3'] = TeamPing3
df['EnemyPing1'] = EnemyPing1
df['EnemyPing2'] = EnemyPing2
df['EnemyPing3'] = EnemyPing3
df['TeamDamageReceived1'] = TeamDamageReceived1
df['TeamDamageReceived2'] = TeamDamageReceived2
df['TeamDamageReceived3'] = TeamDamageReceived3
df['EnemyDamageReceived1'] = EnemyDamageReceived1
df['EnemyDamageReceived2'] = EnemyDamageReceived2
df['EnemyDamageReceived3'] = EnemyDamageReceived3
df['TeamTotalScore1'] = TeamTotalScore1
df['TeamTotalScore2'] = TeamTotalScore2
df['TeamTotalScore3'] = TeamTotalScore3
df['EnemyTotalScore1'] = EnemyTotalScore1
df['EnemyTotalScore2'] = EnemyTotalScore2
df['EnemyTotalScore3'] = EnemyTotalScore3
df['TeamTimeAlive1'] = TeamTimeAlive1
df['TeamTimeAlive2'] = TeamTimeAlive2
df['TeamTimeAlive3'] = TeamTimeAlive3
df['EnemyTimeAlive1'] = EnemyTimeAlive1
df['EnemyTimeAlive2'] = EnemyTimeAlive2
df['EnemyTimeAlive3'] = EnemyTimeAlive3
df['TeamScoreDamageRatio1'] = TeamScoreDamageRatio1
df['TeamScoreDamageRatio2'] = TeamScoreDamageRatio2
df['TeamScoreDamageRatio3'] = TeamScoreDamageRatio3
df['EnemyScoreDamageRatio1'] = EnemyScoreDamageRatio1
df['EnemyScoreDamageRatio2'] = EnemyScoreDamageRatio2
df['EnemyScoreDamageRatio3'] = EnemyScoreDamageRatio3
df['TeamScorePerSecond1'] = TeamScorePerSecond1
df['TeamScorePerSecond2'] = TeamScorePerSecond2
df['TeamScorePerSecond3'] = TeamScorePerSecond3
df['EnemyScorePerSecond1'] = EnemyScorePerSecond1
df['EnemyScorePerSecond2'] = EnemyScorePerSecond2
df['EnemyScorePerSecond3'] = EnemyScorePerSecond3
df['TeamTotalTimePlayed1'] = TeamTotalTimePlayed1
df['TeamTotalTimePlayed2'] = TeamTotalTimePlayed2
df['TeamTotalTimePlayed3'] = TeamTotalTimePlayed3
df['EnemyTotalTimePlayed1'] = EnemyTotalTimePlayed1
df['EnemyTotalTimePlayed2'] = EnemyTotalTimePlayed2
df['EnemyTotalTimePlayed3'] = EnemyTotalTimePlayed3
df['TeamChampionTimePlayed1'] = TeamChampionTimePlayed1
df['TeamChampionTimePlayed2'] = TeamChampionTimePlayed2
df['TeamChampionTimePlayed3'] = TeamChampionTimePlayed3
df['EnemyChampionTimePlayed1'] = EnemyChampionTimePlayed1
df['EnemyChampionTimePlayed2'] = EnemyChampionTimePlayed2
df['EnemyChampionTimePlayed3'] = EnemyChampionTimePlayed3
df['TeamLeagueDivision1'] = TeamLeagueDivision1
df['TeamLeagueDivision2'] = TeamLeagueDivision2
df['TeamLeagueDivision3'] = TeamLeagueDivision3
df['EnemyLeagueDivision1'] = EnemyLeagueDivision1
df['EnemyLeagueDivision2'] = EnemyLeagueDivision2
df['EnemyLeagueDivision3'] = EnemyLeagueDivision3
df['TeamChampion1'] = TeamChampion1
df['TeamChampion2'] = TeamChampion2
df['TeamChampion3'] = TeamChampion3
df['EnemyChampion1'] = EnemyChampion1
df['EnemyChampion2'] = EnemyChampion2
df['EnemyChampion3'] = EnemyChampion3
df['TeamRole1'] = TeamRole1
df['TeamRole2'] = TeamRole2
df['TeamRole3'] = TeamRole3
df['EnemyRole1'] = EnemyRole1
df['EnemyRole2'] = EnemyRole2
df['EnemyRole3'] = EnemyRole3
df['TeamRoleDetail1'] = TeamRoleDetail1
df['TeamRoleDetail2'] = TeamRoleDetail2
df['TeamRoleDetail3'] = TeamRoleDetail3
df['EnemyRoleDetail1'] = EnemyRoleDetail1
df['EnemyRoleDetail2'] = EnemyRoleDetail2
df['EnemyRoleDetail3'] = EnemyRoleDetail3

In [26]:
#Create League Division mapping to be able to compute average leagues between players
values = [0, 1, 2, 3, 4, 5,
          6, 7, 8, 9, 10,
          11, 12, 13, 14, 15,
          16, 17, 18, 19, 20,
          21, 22, 23, 24, 25,
          26, 27, 28, 29, 30]

league_div = ['Bronze 0', 'Bronze 5', 'Bronze 4', 'Bronze 3', 'Bronze 2', 'Bronze 1',
              'Silver 5', 'Silver 4', 'Silver 3', 'Silver 2', 'Silver 1',
              'Gold 5', 'Gold 4', 'Gold 3', 'Gold 2', 'Gold 1',
              'Platinum 5', 'Platinum 4', 'Platinum 3', 'Platinum 2', 'Platinum 1',
              'Diamond 5', 'Diamond 4', 'Diamond 3', 'Diamond 2', 'Diamond 1',
              'Champion 5', 'Champion 4', 'Champion 3', 'Champion 2', 'Champion 1']

League_Div = pd.DataFrame(list(zip(league_div, values)), columns = ['League Division', 'Value'])

In [27]:
#Find league division mappings for related columns
side = ['Team', 'Enemy']
for num in ['1', '2', '3']:
    df[side[0]+'LeagueDivisionValue'+num] = np.NaN
    df[side[1]+'LeagueDivisionValue'+num] = np.NaN

for index, row in df.iterrows():
    for num in ['1', '2', '3']:
        if row['Server_Type'] != 'QUICK3V3' and num == '3':
            continue
        for s in side:
            leaguediv = row[s+'LeagueDivision'+num]
            leaguedivValue = League_Div[League_Div['League Division'] == leaguediv]['Value'].values[0]
            df.loc[index, s+'LeagueDivisionValue'+num] = leaguedivValue
    

In [36]:
#Compute net metrics between the rank ordered metrics
side = ['Team', 'Enemy']

for metric in ['Damage', 'Protection', 'ProtectionReceived', 'Control', 'ControlReceived', 'EnergyUsed',
               'EnergyGained', 'Ping', 'DamageReceived', 'TotalScore', 'TimeAlive', 'ScoreDamageRatio',
               'ScorePerSecond', 'TotalTimePlayed', 'ChampionTimePlayed', 'LeagueDivisionValue']:
    
    for num in ['1', '2', '3']:
        
        df['Net'+metric+num] = df[side[0]+metric+num].fillna(0) - df[side[1]+metric+num].fillna(0)

In [29]:
#Find draw rounds
df['Round_Draw'] = 0
for gameid in df['Game_ID'].unique():
    gameDf = df[df['Game_ID'] == gameid]
    for round_num in gameDf['Round'].unique():
        roundDf = gameDf[gameDf['Round'] == round_num]
        if roundDf['Round_Won'].sum() == 0:
            for index, row in roundDf.iterrows():
                df.loc[index, 'Round_Draw'] = 1

In [30]:
#Find final round scores for each game
df['Game_Score'] = ''
for gameid in df['Game_ID'].unique():
    gameDf = df[df['Game_ID'] == gameid]
    users_list = []
    for index, row in gameDf.iterrows():
        if row['User_ID'] not in users_list:
            user_rounds = gameDf[gameDf['User_ID'] == row['User_ID']]
            user_rounds_nodraw = user_rounds[user_rounds['Round_Draw'] == 0]
            user_round_won = sum(user_rounds_nodraw['Round_Won'] == 1)
            user_round_lost = sum(user_rounds_nodraw['Round_Won'] == 0)

            for index2, row2 in user_rounds.iterrows():
                df.loc[index2, 'Game_Score'] = str(user_round_won) + '-' + str(user_round_lost)

            users_list.append(row['User_ID'])

In [31]:
#Pool battlerite types into team / enemy team
def add_team_battlerite_types(x):
    '''
    Create new column containing all battlerite types from the team.
    '''
    if x['Server_Type'] == 'QUICK3V3':
        return (x['BattleritesType'] + ', ' + x['Ally1BattleritesType'] + ', ' + x['Ally2BattleritesType'])
    
    else:
        return(x['BattleritesType'] + ', ' + x['Ally1BattleritesType'])
    
def add_enemy_battlerite_types(x):
    '''
    Create new column containing all enemy battlerite types.
    '''
    if x['Server_Type'] == 'QUICK3V3':
        return (x['Enemy1BattleritesType'] + ', ' + x['Enemy2BattleritesType'] + ', ' + x['Enemy3BattleritesType'])
    
    else:
        return(x['Enemy1BattleritesType'] + ', ' + x['Enemy2BattleritesType'])

df['TeamBattleritesType'] = df.apply(add_team_battlerite_types, axis = 1)
df['EnemyBattleritesType'] = df.apply(add_enemy_battlerite_types, axis = 1)

In [32]:
#Standardize values for order differences for same set
standardize_values_dupe(df, 'TeamBattleritesType')
standardize_values_dupe(df, 'EnemyBattleritesType')

In [33]:
def remove_dupe_battlerites(col):
    '''
    Reduce battlerites type list more to only count each value once. Without this step still over 7000 levels exist.
    '''
    new_list = []
    for row in df[col]:
        new_list.append(', '.join(list(set(row.split(', ')))))
    df[col] = new_list

remove_dupe_battlerites('TeamBattleritesType')
remove_dupe_battlerites('EnemyBattleritesType')
standardize_values_dupe(df, 'TeamBattleritesType')
standardize_values_dupe(df, 'EnemyBattleritesType')

In [34]:
df.head(1)

,Game_ID,Map,Match_Type,Ranking_Type,Server_Type,Solo_Queue,Region,Limited_MM,Name,User_ID,Champion,Role,Group,League,Division,Total_Time_Played,Account_Level,Champion_Level,Champion_Time_Played,Round,Kills,Deaths,Assists,Damage,Damage_Received,Protection,Protection_Received,Control,Control_Received,Energy_Used,Energy_Gained,Abilities_Used,Ultimates,Num_R,Num_EX1,Num_EX2,Num_Energy_Used,Orb_Kills,First_Orb,MVP,Round_Length,Round_Won,Total_Score,Highest_Score,Health_Shards,Energy_Shards,Team_Orbs,Enemy_Orbs,Time_Alive,Premade,Queue_Time,Ping,Avatar,Title,Outfit Rarity,Outfit,Attachment Rarity,Attachment,Pose Rarity,Pose,Mount Rarity,Mount,Battlerite 1,Battlerite 2,Battlerite 3,Battlerite 4,Battlerite 5,Battlerites,Team_Comp,Team_Roles,Ally1Role,Ally1Champion,Ally1ChampionPlayed,Ally1TotalPlayed,Ally1TotalScore,Ally1DamageReceived,Ally1TimeAlive,Ally1League,Ally1Ping,Ally1NumEnergyUsed,Ally1HighestScore,Ally1Battlerite1,Ally1Battlerite2,Ally1Battlerite3,Ally1Battlerite4,Ally1Battlerite5,Ally1Battlerites,Ally2Role,Ally2Champion,Ally2ChampionPlayed,Ally2TotalPlayed,Ally2TotalScore,Ally2DamageReceived,Ally2TimeAlive,Ally2League,Ally2Ping,Ally2NumEnergyUsed,Ally2HighestScore,Ally2Battlerite1,Ally2Battlerite2,Ally2Battlerite3,Ally2Battlerite4,Ally2Battlerite5,Ally2Battlerites,Enemy_Comp,Enemy_Roles,Enemy_Total_HealthShards,Enemy_Total_EnergyShards,Enemy1Role,Enemy1Champion,Enemy1ChampionPlayed,Enemy1TotalPlayed,Enemy1TotalScore,Enemy1DamageReceived,Enemy1TimeAlive,Enemy1League,Enemy1Ping,Enemy1NumEnergyUsed,Enemy1HighestScore,Enemy1Battlerite1,Enemy1Battlerite2,Enemy1Battlerite3,Enemy1Battlerite4,Enemy1Battlerite5,Enemy1Battlerites,Enemy2Role,Enemy2Champion,Enemy2ChampionPlayed,Enemy2TotalPlayed,Enemy2TotalScore,Enemy2DamageReceived,Enemy2TimeAlive,Enemy2League,Enemy2Ping,Enemy2NumEnergyUsed,Enemy2HighestScore,Enemy3Role,Enemy3Champion,Enemy3ChampionPlayed,Enemy3TotalPlayed,Enemy3TotalScore,Enemy3DamageReceived,Enemy3TimeAlive,Enemy3League,Enemy3Ping,Enemy3NumEnergyUsed,Enemy3HighestScore,Enemy2Battlerite1,Enemy2Battlerite2,Enemy2Battlerite3,Enemy2Battlerite4,Enemy2Battlerite5,Enemy2Battlerites,Enemy3Battlerite1,Enemy3Battlerite2,Enemy3Battlerite3,Enemy3Battlerite4,Enemy3Battlerite5,Enemy3Battlerites,Role_Detail,Ally1RoleDetail,Ally2RoleDetail,Enemy1RoleDetail,Enemy2RoleDetail,Enemy3RoleDetail,TeamRolesDetail,EnemyRolesDetail,BattleritesType,Ally1BattleritesType,Ally2BattleritesType,Enemy1BattleritesType,Enemy2BattleritesType,Enemy3BattleritesType,ScoreDamageRatio,Ally1ScoreDamageRatio,Ally2ScoreDamageRatio,Enemy1ScoreDamageRatio,Enemy2ScoreDamageRatio,Enemy3ScoreDamageRatio,ScorePerSecond,Ally1ScorePerSecond,Ally2ScorePerSecond,Enemy1ScorePerSecond,Enemy2ScorePerSecond,Enemy3ScorePerSecond,TeamTotalHealthShards,TeamTotalEnergyShards,Ally1Division,Ally1Damage,Ally1Protection,Ally1ProtectionReceived,Ally1Control,Ally1ControlReceived,Ally1EnergyUsed,Ally1EnergyGained,Ally1Deaths,Ally2Division,Ally2Damage,Ally2Protection,Ally2ProtectionReceived,Ally2Control,Ally2ControlReceived,Ally2EnergyUsed,Ally2EnergyGained,Ally2Deaths,Enemy1Division,Enemy1Damage,Enemy1Protection,Enemy1ProtectionReceived,Enemy1Control,Enemy1ControlReceived,Enemy1EnergyUsed,Enemy1EnergyGained,Enemy1Deaths,Enemy2Division,Enemy2Damage,Enemy2Protection,Enemy2ProtectionReceived,Enemy2Control,Enemy2ControlReceived,Enemy2EnergyUsed,Enemy2EnergyGained,Enemy2Deaths,Enemy3Division,Enemy3Damage,Enemy3Protection,Enemy3ProtectionReceived,Enemy3Control,Enemy3ControlReceived,Enemy3EnergyUsed,Enemy3EnergyGained,Enemy3Deaths,TotalDeaths,LeagueDivision,Ally1LeagueDivision,Ally2LeagueDivision,Enemy1LeagueDivision,Enemy2LeagueDivision,Enemy3LeagueDivision,TeamRatioUltimatesAll20,EnemyRatioUltimatesAll20,TeamRatioREX50,EnemyRatioREX50,NetHealthShards,NetEnergyShards,NetOrbs,NetDeathOrbs,TeamNetUltimates,TeamNetEX,TeamNetR,TeamDamage1,TeamDamage2,TeamDamage3,EnemyDamage1,EnemyDamage2,EnemyDamage3,TeamProtection1,TeamProtection2,TeamProtection3,EnemyProtection1,EnemyProtection2,EnemyProte

In [66]:
#Write data to file in pieces for GitHub file limit purposes
step = 0
for i in range(m.ceil(len(df)/20000)):
    if i == 0:
        df.loc[step:20000+step, :].to_csv('Processed_Data' + '_' + str(i+1) + '.csv', index = False)
    else:
        df.loc[step+1:20000+step, :].to_csv('Processed_Data' + '_' + str(i+1) + '.csv', index = False)
    step += 20000

In [71]:
df['TeamRatioREX50'].value_counts()

1    153316
0     64744
Name: TeamRatioREX50, dtype: int64